    \item k-UpCCGSD ansatz with $k\in\{1; 2; 3\}$, L-BFGS-B optimizer, Jordan-Wigner mapping, Aer simulator,

        \item circuit depth,
    \item gate fidelity,
    \item parameter count,
    \item TRAINABILITY,
    \item EXPRESSIBILITY.

     $H_2$, $He$, $LiH$, $BeH_2$ molecules.
     
     \item converged ground state energy graph, depending on the inter-atomic distance,
\item accuracy, with reference of FCI calculations, absolute error (\parencite{sims2006high}), depending on the inter-atomic distance,
\item number of iterations needed to achieve convergence (gradient tolerance is $10^{-5}$), depending on the inter-atomic distance,
\item for one inter-atomic distance, graph of convergence,
\item runtime, depending on the inter-atomic distance.

> Data from kUpCCGSD-tequila-env.ipynb, or to be precise: kupccgsd-hamiltonians.txt and kupccgsd-ansatzes.txt

In [1]:
import re
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate

In [ ]:
molecules = [
    ("H2", lambda d: f"H 0.0 0.0 0.0\nH 0.0 0.0 {d}"),
    ("LiH", lambda d: f"Li 0.0 0.0 0.0\nH 0.0 0.0 {d}"),
    ("BeH2", lambda d: f"Be 0.0 0.0 0.0\nH 0.0 0.0 {d}\nH 0.0 0.0 {-d}"),
]

distances=[0.1, 0.2, 0.5, 0.735, 1.0, 1.5, 2.0, 3.0, 4.0]

ks=[1,2,3]

In [ ]:
hamiltonians=[]
hamiltonian_prep_times=[]

ansatzes=[]
ansatz_prep_times=[]

In [ ]:
# example

from qiskit.quantum_info import SparsePauliOp
import re

def parse_hamiltonian(hamiltonian_str):
    # hamiltonian_str = """
    # +4.9081+0.2909Z(0)-0.0056Y(0)Z(1)Y(2)-0.0056X(0)Z(1)X(2)+0.2909Z(1)-0.0056Y(1)Z(2)Y(3)-0.0056X(1)Z(2)X(3)-0.7411Z(2)-0.7411Z(3)+0.2481Z(0)Z(1)+0.0781Y(0)Y(2)+0.0781X(0)X(2)+0.0781Z(0)Y(1)Z(2)Y(3)+0.0781Z(0)X(1)Z(2)X(3)+0.0629Y(0)X(1)X(2)Y(3)-0.0629Y(0)Y(1)X(2)X(3)-0.0629X(0)X(1)Y(2)Y(3)+0.0629X(0)Y(1)Y(2)X(3)+0.1693Z(0)Z(2)+0.2322Z(0)Z(3)+0.0836Y(0)Z(1)Y(2)Z(3)+0.0836X(0)Z(1)X(2)Z(3)+0.2322Z(1)Z(2)+0.0836Y(1)Y(3)+0.0836X(1)X(3)+0.1693Z(1)Z(3)+0.2329Z(2)Z(3)
    # """
    pattern = r'([+-]?\d+\.\d+)([XYZ\(\)\d]*)'
    matches = re.findall(pattern, hamiltonian_str)

    paulis = []
    coeffs = []

    for coeff, ops in matches:
        max_idx = max(int(i) for _, i in re.findall(r'([XYZ])\((\d)\)', hamiltonian_str))
        qubit_ops = ['I'] * (max_idx + 1)
        for op, idx in re.findall(r'([XYZ])\((\d)\)', ops):
            qubit_ops[int(idx)] = op
        paulis.append(''.join(reversed(qubit_ops)))  # Qiskit uses little-endian
        coeffs.append(complex(float(coeff)))

    sparse_op = SparsePauliOp(paulis, coeffs)
    return sparse_op

hamiltonian_str = """
+4.9081+0.2909Z(0)-0.0056Y(0)Z(1)Y(2)-0.0056X(0)Z(1)X(2)+0.2909Z(1)-0.0056Y(1)Z(2)Y(3)-0.0056X(1)Z(2)X(3)-0.7411Z(2)-0.7411Z(3)+0.2481Z(0)Z(1)+0.0781Y(0)Y(2)+0.0781X(0)X(2)+0.0781Z(0)Y(1)Z(2)Y(3)+0.0781Z(0)X(1)Z(2)X(3)+0.0629Y(0)X(1)X(2)Y(3)-0.0629Y(0)Y(1)X(2)X(3)-0.0629X(0)X(1)Y(2)Y(3)+0.0629X(0)Y(1)Y(2)X(3)+0.1693Z(0)Z(2)+0.2322Z(0)Z(3)+0.0836Y(0)Z(1)Y(2)Z(3)+0.0836X(0)Z(1)X(2)Z(3)+0.2322Z(1)Z(2)+0.0836Y(1)Y(3)+0.0836X(1)X(3)+0.1693Z(1)Z(3)+0.2329Z(2)Z(3)
"""
print(parse_hamiltonian(hamiltonian_str))



SparsePauliOp(['IIII', 'IIIZ', 'IYZY', 'IXZX', 'IIZI', 'YZYI', 'XZXI', 'IZII', 'ZIII', 'IIZZ', 'IYIY', 'IXIX', 'YZYZ', 'XZXZ', 'YXXY', 'XXYY', 'YYXX', 'XYYX', 'IZIZ', 'ZIIZ', 'ZYZY', 'ZXZX', 'IZZI', 'YIYI', 'XIXI', 'ZIZI', 'ZZII'],
              coeffs=[ 4.9081+0.j,  0.2909+0.j, -0.0056+0.j, -0.0056+0.j,  0.2909+0.j,
 -0.0056+0.j, -0.0056+0.j, -0.7411+0.j, -0.7411+0.j,  0.2481+0.j,
  0.0781+0.j,  0.0781+0.j,  0.0781+0.j,  0.0781+0.j,  0.0629+0.j,
 -0.0629+0.j, -0.0629+0.j,  0.0629+0.j,  0.1693+0.j,  0.2322+0.j,
  0.0836+0.j,  0.0836+0.j,  0.2322+0.j,  0.0836+0.j,  0.0836+0.j,
  0.1693+0.j,  0.2329+0.j])


In [9]:
import re

data = """
finished after 21.832292079925537s

molecule: H2, distance: 0.1
+4.9081+0.2909Z(0)-0.0056Y(0)Z(1)Y(2)-0.0056X(0)Z(1)X(2)+0.2909Z(1)-0.0056Y(1)Z(2)Y(3)-0.0056X(1)Z(2)X(3)-0.7411Z(2)-0.7411Z(3)+0.2481Z(0)Z(1)+0.0781Y(0)Y(2)+0.0781X(0)X(2)+0.0781Z(0)Y(1)Z(2)Y(3)+0.0781Z(0)X(1)Z(2)X(3)+0.0629Y(0)X(1)X(2)Y(3)-0.0629Y(0)Y(1)X(2)X(3)-0.0629X(0)X(1)Y(2)Y(3)+0.0629X(0)Y(1)Y(2)X(3)+0.1693Z(0)Z(2)+0.2322Z(0)Z(3)+0.0836Y(0)Z(1)Y(2)Z(3)+0.0836X(0)Z(1)X(2)Z(3)+0.2322Z(1)Z(2)+0.0836Y(1)Y(3)+0.0836X(1)X(3)+0.1693Z(1)Z(3)+0.2329Z(2)Z(3)
finished after 35.75117349624634s

molecule: LiH, distance: 0.1
+2.5983+0.0986Z(0)-0.0000Y(0)Z(1)Y(2)-0.0000X(0)Z(1)X(2)+0.0986Z(1)-0.0000Y(1)Z(2)Y(3)-0.0000X(1)Z(2)X(3)+0.0003Z(2)+0.0003Z(3)+0.0834Z(0)Z(1)+0.0007Y(0)Y(2)+0.0007X(0)X(2)+0.0007Z(0)Y(1)Z(2)Y(3)+0.0007Z(0)X(1)Z(2)X(3)+0.0197Y(0)X(1)X(2)Y(3)-0.0197Y(0)Y(1)X(2)X(3)-0.0197X(0)X(1)Y(2)Y(3)+0.0197X(0)Y(1)Y(2)X(3)+0.0679Z(0)Z(2)+0.0875Z(0)Z(3)+0.0008Y(0)Z(1)Y(2)Z(3)+0.0008X(0)Z(1)X(2)Z(3)+0.0875Z(1)Z(2)+0.0008Y(1)Y(3)+0.0008X(1)X(3)+0.0679Z(1)Z(3)+0.0997Z(2)Z(3)
finished after 66.45494198799133s

molecule: BeH2, distance: 0.1
+14.9486+0.1625Z(0)-0.0777Y(0)Z(1)Y(2)-0.0777X(0)Z(1)X(2)+0.1625Z(1)-0.0777Y(1)Z(2)Y(3)-0.0777X(1)Z(2)X(3)+0.1625Z(2)+0.1625Z(3)-0.2824Z(4)-0.2915Y(4)Z(5)Y(6)-0.2915X(4)Z(5)X(6)-0.2824Z(5)-0.2915Y(5)Z(6)Y(7)-0.2915X(5)Z(6)X(7)-0.2824Z(6)-0.2824Z(7)+0.1671Z(0)Z(1)+0.0067Y(0)Y(2)+0.0067X(0)X(2)+0.0067Z(0)Y(1)Z(2)Y(3)+0.0067Z(0)X(1)Z(2)X(3)+0.0033Y(0)X(1)X(2)Y(3)-0.0033Y(0)Y(1)X(2)X(3)-0.0033X(0)X(1)Y(2)Y(3)+0.0033X(0)Y(1)Y(2)X(3)+0.0364Y(0)X(1)X(4)Y(5)-0.0364Y(0)Y(1)X(4)X(5)-0.0364X(0)X(1)Y(4)Y(5)+0.0364X(0)Y(1)Y(4)X(5)-0.0011Y(0)X(1)X(5)Y(6)-0.0011Y(0)Y(1)Y(5)Y(6)-0.0011X(0)X(1)X(5)X(6)-0.0011X(0)Y(1)Y(5)X(6)+0.0011Y(0)X(1)X(4)Z(5)Z(6)Y(7)-0.0011Y(0)Y(1)X(4)Z(5)Z(6)X(7)-0.0011X(0)X(1)Y(4)Z(5)Z(6)Y(7)+0.0011X(0)Y(1)Y(4)Z(5)Z(6)X(7)+0.0014Y(0)X(1)X(6)Y(7)-0.0014Y(0)Y(1)X(6)X(7)-0.0014X(0)X(1)Y(6)Y(7)+0.0014X(0)Y(1)Y(6)X(7)+0.1068Z(0)Z(2)+0.0116Y(0)Z(1)X(2)X(4)Z(5)Y(6)-0.0001Y(0)Z(1)Y(2)Y(4)Z(5)Y(6)-0.0117Y(0)Z(1)Y(2)X(4)Z(5)X(6)-0.0117X(0)Z(1)X(2)Y(4)Z(5)Y(6)-0.0001X(0)Z(1)X(2)X(4)Z(5)X(6)+0.0116X(0)Z(1)Y(2)Y(4)Z(5)X(6)+0.1101Z(0)Z(3)+0.0067Y(0)Z(1)Y(2)Z(3)+0.0067X(0)Z(1)X(2)Z(3)+0.0026Y(0)Z(1)Z(2)X(3)X(4)Y(5)-0.0026Y(0)Z(1)Z(2)Y(3)X(4)X(5)-0.0026X(0)Z(1)Z(2)X(3)Y(4)Y(5)+0.0026X(0)Z(1)Z(2)Y(3)Y(4)X(5)-0.0011Y(0)Z(1)Z(2)X(3)X(5)Y(6)-0.0011Y(0)Z(1)Z(2)Y(3)Y(5)Y(6)-0.0011X(0)Z(1)Z(2)X(3)X(5)X(6)-0.0011X(0)Z(1)Z(2)Y(3)Y(5)X(6)+0.0127Y(0)Z(1)Z(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0127Y(0)Z(1)Z(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0127X(0)Z(1)Z(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0127X(0)Z(1)Z(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0026Y(0)Z(1)Z(2)X(3)X(6)Y(7)-0.0026Y(0)Z(1)Z(2)Y(3)X(6)X(7)-0.0026X(0)Z(1)Z(2)X(3)Y(6)Y(7)+0.0026X(0)Z(1)Z(2)Y(3)Y(6)X(7)+0.1275Z(0)Z(4)-0.0015Z(0)Y(4)Z(5)Y(6)-0.0015Z(0)X(4)Z(5)X(6)+0.0067Y(0)Z(1)Y(2)Z(4)+0.0067X(0)Z(1)X(2)Z(4)+0.1639Z(0)Z(5)+0.0041Y(0)Z(1)Y(2)Z(5)+0.0041X(0)Z(1)X(2)Z(5)-0.0026Z(0)Y(5)Z(6)Y(7)-0.0026Z(0)X(5)Z(6)X(7)+0.0010Y(0)Z(1)Y(2)Y(5)Z(6)Y(7)+0.0010Y(0)Z(1)Y(2)X(5)Z(6)X(7)+0.0010X(0)Z(1)X(2)Y(5)Z(6)Y(7)+0.0010X(0)Z(1)X(2)X(5)Z(6)X(7)+0.1100Z(0)Z(6)+0.0067Y(0)Z(1)Y(2)Z(6)+0.0067X(0)Z(1)X(2)Z(6)+0.1115Z(0)Z(7)+0.0041Y(0)Z(1)Y(2)Z(7)+0.0041X(0)Z(1)X(2)Z(7)+0.1101Z(1)Z(2)+0.0067Y(1)Y(3)+0.0067X(1)X(3)-0.0026Y(1)X(2)X(4)Y(5)-0.0026Y(1)Y(2)Y(4)Y(5)-0.0026X(1)X(2)X(4)X(5)-0.0026X(1)Y(2)Y(4)X(5)-0.0011Y(1)X(2)X(4)Z(5)Z(6)Y(7)-0.0011Y(1)Y(2)Y(4)Z(5)Z(6)Y(7)-0.0011X(1)X(2)X(4)Z(5)Z(6)X(7)-0.0011X(1)Y(2)Y(4)Z(5)Z(6)X(7)+0.0127Y(1)X(2)X(5)Y(6)-0.0127Y(1)Y(2)X(5)X(6)-0.0127X(1)X(2)Y(5)Y(6)+0.0127X(1)Y(2)Y(5)X(6)-0.0026Y(1)X(2)X(6)Y(7)-0.0026Y(1)Y(2)Y(6)Y(7)-0.0026X(1)X(2)X(6)X(7)-0.0026X(1)Y(2)Y(6)X(7)+0.1068Z(1)Z(3)+0.0116Y(1)Z(2)X(3)X(5)Z(6)Y(7)-0.0001Y(1)Z(2)Y(3)Y(5)Z(6)Y(7)-0.0117Y(1)Z(2)Y(3)X(5)Z(6)X(7)-0.0117X(1)Z(2)X(3)Y(5)Z(6)Y(7)-0.0001X(1)Z(2)X(3)X(5)Z(6)X(7)+0.0116X(1)Z(2)Y(3)Y(5)Z(6)X(7)+0.1639Z(1)Z(4)+0.0041Y(1)Z(2)Y(3)Z(4)+0.0041X(1)Z(2)X(3)Z(4)-0.0026Z(1)Y(4)Z(5)Y(6)-0.0026Z(1)X(4)Z(5)X(6)+0.0010Y(1)Z(2)Y(3)Y(4)Z(5)Y(6)+0.0010Y(1)Z(2)Y(3)X(4)Z(5)X(6)+0.0010X(1)Z(2)X(3)Y(4)Z(5)Y(6)+0.0010X(1)Z(2)X(3)X(4)Z(5)X(6)+0.1275Z(1)Z(5)-0.0015Z(1)Y(5)Z(6)Y(7)-0.0015Z(1)X(5)Z(6)X(7)+0.0067Y(1)Z(2)Y(3)Z(5)+0.0067X(1)Z(2)X(3)Z(5)+0.1115Z(1)Z(6)+0.0041Y(1)Z(2)Y(3)Z(6)+0.0041X(1)Z(2)X(3)Z(6)+0.1100Z(1)Z(7)+0.0067Y(1)Z(2)Y(3)Z(7)+0.0067X(1)Z(2)X(3)Z(7)+0.1671Z(2)Z(3)+0.0014Y(2)X(3)X(4)Y(5)-0.0014Y(2)Y(3)X(4)X(5)-0.0014X(2)X(3)Y(4)Y(5)+0.0014X(2)Y(3)Y(4)X(5)-0.0011Y(2)X(3)X(5)Y(6)-0.0011Y(2)Y(3)Y(5)Y(6)-0.0011X(2)X(3)X(5)X(6)-0.0011X(2)Y(3)Y(5)X(6)+0.0011Y(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0011Y(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0011X(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0011X(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0364Y(2)X(3)X(6)Y(7)-0.0364Y(2)Y(3)X(6)X(7)-0.0364X(2)X(3)Y(6)Y(7)+0.0364X(2)Y(3)Y(6)X(7)+0.1100Z(2)Z(4)-0.0015Z(2)Y(4)Z(5)Y(6)-0.0015Z(2)X(4)Z(5)X(6)+0.1115Z(2)Z(5)-0.0026Z(2)Y(5)Z(6)Y(7)-0.0026Z(2)X(5)Z(6)X(7)+0.1275Z(2)Z(6)+0.1639Z(2)Z(7)+0.1115Z(3)Z(4)-0.0026Z(3)Y(4)Z(5)Y(6)-0.0026Z(3)X(4)Z(5)X(6)+0.1100Z(3)Z(5)-0.0015Z(3)Y(5)Z(6)Y(7)-0.0015Z(3)X(5)Z(6)X(7)+0.1639Z(3)Z(6)+0.1275Z(3)Z(7)+0.1763Z(4)Z(5)-0.0039Y(4)Y(6)-0.0039X(4)X(6)-0.0039Z(4)Y(5)Z(6)Y(7)-0.0039Z(4)X(5)Z(6)X(7)+0.0027Y(4)X(5)X(6)Y(7)-0.0027Y(4)Y(5)X(6)X(7)-0.0027X(4)X(5)Y(6)Y(7)+0.0027X(4)Y(5)Y(6)X(7)+0.1140Z(4)Z(6)+0.1167Z(4)Z(7)-0.0039Y(4)Z(5)Y(6)Z(7)-0.0039X(4)Z(5)X(6)Z(7)+0.1167Z(5)Z(6)-0.0039Y(5)Y(7)-0.0039X(5)X(7)+0.1140Z(5)Z(7)+0.1763Z(6)Z(7)
finished after 20.54081106185913s

molecule: H2, distance: 0.2
+2.2347+0.2729Z(0)-0.0043Y(0)Z(1)Y(2)-0.0043X(0)Z(1)X(2)+0.2729Z(1)-0.0043Y(1)Z(2)Y(3)-0.0043X(1)Z(2)X(3)-0.6693Z(2)-0.6693Z(3)+0.2328Z(0)Z(1)+0.0716Y(0)Y(2)+0.0716X(0)X(2)+0.0716Z(0)Y(1)Z(2)Y(3)+0.0716Z(0)X(1)Z(2)X(3)+0.0571Y(0)X(1)X(2)Y(3)-0.0571Y(0)Y(1)X(2)X(3)-0.0571X(0)X(1)Y(2)Y(3)+0.0571X(0)Y(1)Y(2)X(3)+0.1601Z(0)Z(2)+0.2172Z(0)Z(3)+0.0760Y(0)Z(1)Y(2)Z(3)+0.0760X(0)Z(1)X(2)Z(3)+0.2172Z(1)Z(2)+0.0760Y(1)Y(3)+0.0760X(1)X(3)+0.1601Z(1)Z(3)+0.2167Z(2)Z(3)
finished after 35.24649500846863s

molecule: LiH, distance: 0.2
-4.1421+0.0929Z(0)+0.0004Y(0)Z(1)Y(2)+0.0004X(0)Z(1)X(2)+0.0929Z(1)+0.0004Y(1)Z(2)Y(3)+0.0004X(1)Z(2)X(3)+0.0127Z(2)+0.0127Z(3)+0.0804Z(0)Z(1)-0.0045Y(0)Y(2)-0.0045X(0)X(2)-0.0045Z(0)Y(1)Z(2)Y(3)-0.0045Z(0)X(1)Z(2)X(3)+0.0191Y(0)X(1)X(2)Y(3)-0.0191Y(0)Y(1)X(2)X(3)-0.0191X(0)X(1)Y(2)Y(3)+0.0191X(0)Y(1)Y(2)X(3)+0.0656Z(0)Z(2)+0.0847Z(0)Z(3)-0.0049Y(0)Z(1)Y(2)Z(3)-0.0049X(0)Z(1)X(2)Z(3)+0.0847Z(1)Z(2)-0.0049Y(1)Y(3)-0.0049X(1)X(3)+0.0656Z(1)Z(3)+0.0969Z(2)Z(3)
finished after 63.92885446548462s

molecule: BeH2, distance: 0.2
-3.1135+0.2024Z(0)+0.0612Y(0)Z(1)Y(2)+0.0612X(0)Z(1)X(2)+0.2024Z(1)+0.0612Y(1)Z(2)Y(3)+0.0612X(1)Z(2)X(3)+0.2024Z(2)+0.2024Z(3)-0.3119Z(4)-0.2923Y(4)Z(5)Y(6)-0.2923X(4)Z(5)X(6)-0.3119Z(5)-0.2923Y(5)Z(6)Y(7)-0.2923X(5)Z(6)X(7)-0.3119Z(6)-0.3119Z(7)+0.1673Z(0)Z(1)-0.0159Y(0)Y(2)-0.0159X(0)X(2)-0.0159Z(0)Y(1)Z(2)Y(3)-0.0159Z(0)X(1)Z(2)X(3)+0.0070Y(0)X(1)X(2)Y(3)-0.0070Y(0)Y(1)X(2)X(3)-0.0070X(0)X(1)Y(2)Y(3)+0.0070X(0)Y(1)Y(2)X(3)+0.0014Y(0)X(1)X(4)Y(5)-0.0014Y(0)Y(1)X(4)X(5)-0.0014X(0)X(1)Y(4)Y(5)+0.0014X(0)Y(1)Y(4)X(5)-0.0005Y(0)X(1)X(5)Y(6)-0.0005Y(0)Y(1)Y(5)Y(6)-0.0005X(0)X(1)X(5)X(6)-0.0005X(0)Y(1)Y(5)X(6)+0.0005Y(0)X(1)X(4)Z(5)Z(6)Y(7)-0.0005Y(0)Y(1)X(4)Z(5)Z(6)X(7)-0.0005X(0)X(1)Y(4)Z(5)Z(6)Y(7)+0.0005X(0)Y(1)Y(4)Z(5)Z(6)X(7)+0.0352Y(0)X(1)X(6)Y(7)-0.0352Y(0)Y(1)X(6)X(7)-0.0352X(0)X(1)Y(6)Y(7)+0.0352X(0)Y(1)Y(6)X(7)+0.1068Z(0)Z(2)+0.0108Y(0)Z(1)X(2)X(4)Z(5)Y(6)+0.0109Y(0)Z(1)Y(2)Y(4)Z(5)Y(6)+0.0002Y(0)Z(1)Y(2)X(4)Z(5)X(6)+0.0002X(0)Z(1)X(2)Y(4)Z(5)Y(6)+0.0109X(0)Z(1)X(2)X(4)Z(5)X(6)+0.0108X(0)Z(1)Y(2)Y(4)Z(5)X(6)+0.1139Z(0)Z(3)-0.0159Y(0)Z(1)Y(2)Z(3)-0.0159X(0)Z(1)X(2)Z(3)-0.0013Y(0)Z(1)Z(2)X(3)X(4)Y(5)+0.0013Y(0)Z(1)Z(2)Y(3)X(4)X(5)+0.0013X(0)Z(1)Z(2)X(3)Y(4)Y(5)-0.0013X(0)Z(1)Z(2)Y(3)Y(4)X(5)+0.0117Y(0)Z(1)Z(2)X(3)X(5)Y(6)+0.0117Y(0)Z(1)Z(2)Y(3)Y(5)Y(6)+0.0117X(0)Z(1)Z(2)X(3)X(5)X(6)+0.0117X(0)Z(1)Z(2)Y(3)Y(5)X(6)-0.0009Y(0)Z(1)Z(2)X(3)X(4)Z(5)Z(6)Y(7)+0.0009Y(0)Z(1)Z(2)Y(3)X(4)Z(5)Z(6)X(7)+0.0009X(0)Z(1)Z(2)X(3)Y(4)Z(5)Z(6)Y(7)-0.0009X(0)Z(1)Z(2)Y(3)Y(4)Z(5)Z(6)X(7)-0.0013Y(0)Z(1)Z(2)X(3)X(6)Y(7)+0.0013Y(0)Z(1)Z(2)Y(3)X(6)X(7)+0.0013X(0)Z(1)Z(2)X(3)Y(6)Y(7)-0.0013X(0)Z(1)Z(2)Y(3)Y(6)X(7)+0.1107Z(0)Z(4)+0.0001Z(0)Y(4)Z(5)Y(6)+0.0001Z(0)X(4)Z(5)X(6)-0.0135Y(0)Z(1)Y(2)Z(4)-0.0135X(0)Z(1)X(2)Z(4)+0.1121Z(0)Z(5)-0.0121Y(0)Z(1)Y(2)Z(5)-0.0121X(0)Z(1)X(2)Z(5)-0.0005Z(0)Y(5)Z(6)Y(7)-0.0005Z(0)X(5)Z(6)X(7)-0.0008Y(0)Z(1)Y(2)Y(5)Z(6)Y(7)-0.0008Y(0)Z(1)Y(2)X(5)Z(6)X(7)-0.0008X(0)Z(1)X(2)Y(5)Z(6)Y(7)-0.0008X(0)Z(1)X(2)X(5)Z(6)X(7)+0.1297Z(0)Z(6)-0.0135Y(0)Z(1)Y(2)Z(6)-0.0135X(0)Z(1)X(2)Z(6)+0.1649Z(0)Z(7)-0.0121Y(0)Z(1)Y(2)Z(7)-0.0121X(0)Z(1)X(2)Z(7)+0.1139Z(1)Z(2)-0.0159Y(1)Y(3)-0.0159X(1)X(3)+0.0013Y(1)X(2)X(4)Y(5)+0.0013Y(1)Y(2)Y(4)Y(5)+0.0013X(1)X(2)X(4)X(5)+0.0013X(1)Y(2)Y(4)X(5)+0.0117Y(1)X(2)X(4)Z(5)Z(6)Y(7)+0.0117Y(1)Y(2)Y(4)Z(5)Z(6)Y(7)+0.0117X(1)X(2)X(4)Z(5)Z(6)X(7)+0.0117X(1)Y(2)Y(4)Z(5)Z(6)X(7)-0.0009Y(1)X(2)X(5)Y(6)+0.0009Y(1)Y(2)X(5)X(6)+0.0009X(1)X(2)Y(5)Y(6)-0.0009X(1)Y(2)Y(5)X(6)+0.0013Y(1)X(2)X(6)Y(7)+0.0013Y(1)Y(2)Y(6)Y(7)+0.0013X(1)X(2)X(6)X(7)+0.0013X(1)Y(2)Y(6)X(7)+0.1068Z(1)Z(3)+0.0108Y(1)Z(2)X(3)X(5)Z(6)Y(7)+0.0109Y(1)Z(2)Y(3)Y(5)Z(6)Y(7)+0.0002Y(1)Z(2)Y(3)X(5)Z(6)X(7)+0.0002X(1)Z(2)X(3)Y(5)Z(6)Y(7)+0.0109X(1)Z(2)X(3)X(5)Z(6)X(7)+0.0108X(1)Z(2)Y(3)Y(5)Z(6)X(7)+0.1121Z(1)Z(4)-0.0121Y(1)Z(2)Y(3)Z(4)-0.0121X(1)Z(2)X(3)Z(4)-0.0005Z(1)Y(4)Z(5)Y(6)-0.0005Z(1)X(4)Z(5)X(6)-0.0008Y(1)Z(2)Y(3)Y(4)Z(5)Y(6)-0.0008Y(1)Z(2)Y(3)X(4)Z(5)X(6)-0.0008X(1)Z(2)X(3)Y(4)Z(5)Y(6)-0.0008X(1)Z(2)X(3)X(4)Z(5)X(6)+0.1107Z(1)Z(5)+0.0001Z(1)Y(5)Z(6)Y(7)+0.0001Z(1)X(5)Z(6)X(7)-0.0135Y(1)Z(2)Y(3)Z(5)-0.0135X(1)Z(2)X(3)Z(5)+0.1649Z(1)Z(6)-0.0121Y(1)Z(2)Y(3)Z(6)-0.0121X(1)Z(2)X(3)Z(6)+0.1297Z(1)Z(7)-0.0135Y(1)Z(2)Y(3)Z(7)-0.0135X(1)Z(2)X(3)Z(7)+0.1673Z(2)Z(3)+0.0352Y(2)X(3)X(4)Y(5)-0.0352Y(2)Y(3)X(4)X(5)-0.0352X(2)X(3)Y(4)Y(5)+0.0352X(2)Y(3)Y(4)X(5)-0.0005Y(2)X(3)X(5)Y(6)-0.0005Y(2)Y(3)Y(5)Y(6)-0.0005X(2)X(3)X(5)X(6)-0.0005X(2)Y(3)Y(5)X(6)+0.0005Y(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0005Y(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0005X(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0005X(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0014Y(2)X(3)X(6)Y(7)-0.0014Y(2)Y(3)X(6)X(7)-0.0014X(2)X(3)Y(6)Y(7)+0.0014X(2)Y(3)Y(6)X(7)+0.1297Z(2)Z(4)+0.0001Z(2)Y(4)Z(5)Y(6)+0.0001Z(2)X(4)Z(5)X(6)+0.1649Z(2)Z(5)-0.0005Z(2)Y(5)Z(6)Y(7)-0.0005Z(2)X(5)Z(6)X(7)+0.1107Z(2)Z(6)+0.1121Z(2)Z(7)+0.1649Z(3)Z(4)-0.0005Z(3)Y(4)Z(5)Y(6)-0.0005Z(3)X(4)Z(5)X(6)+0.1297Z(3)Z(5)+0.0001Z(3)Y(5)Z(6)Y(7)+0.0001Z(3)X(5)Z(6)X(7)+0.1121Z(3)Z(6)+0.1107Z(3)Z(7)+0.1792Z(4)Z(5)-0.0014Y(4)Y(6)-0.0014X(4)X(6)-0.0014Z(4)Y(5)Z(6)Y(7)-0.0014Z(4)X(5)Z(6)X(7)+0.0020Y(4)X(5)X(6)Y(7)-0.0020Y(4)Y(5)X(6)X(7)-0.0020X(4)X(5)Y(6)Y(7)+0.0020X(4)Y(5)Y(6)X(7)+0.1129Z(4)Z(6)+0.1149Z(4)Z(7)-0.0014Y(4)Z(5)Y(6)Z(7)-0.0014X(4)Z(5)X(6)Z(7)+0.1149Z(5)Z(6)-0.0014Y(5)Y(7)-0.0014X(5)X(7)+0.1129Z(5)Z(7)+0.1792Z(6)Z(7)
finished after 18.6542911529541s

molecule: H2, distance: 0.5
+0.4651+0.1965Z(0)+0.1965Z(1)-0.4321Z(2)-0.4321Z(3)+0.1897Z(0)Z(1)+0.0447Y(0)X(1)X(2)Y(3)-0.0447Y(0)Y(1)X(2)X(3)-0.0447X(0)X(1)Y(2)Y(3)+0.0447X(0)Y(1)Y(2)X(3)+0.1448Z(0)Z(2)+0.1895Z(0)Z(3)+0.1895Z(1)Z(2)+0.1448Z(1)Z(3)+0.2060Z(2)Z(3)
finished after 44.27661323547363s

molecule: LiH, distance: 0.5
-6.6084+0.0815Z(0)-0.0072Y(0)Z(1)Y(2)-0.0072X(0)Z(1)X(2)+0.0815Z(1)-0.0072Y(1)Z(2)Y(3)-0.0072X(1)Z(2)X(3)-0.1374Z(2)-0.1374Z(3)+0.1164Z(0)Z(1)+0.0364Y(0)Y(2)+0.0364X(0)X(2)+0.0364Z(0)Y(1)Z(2)Y(3)+0.0364Z(0)X(1)Z(2)X(3)+0.0313Y(0)X(1)X(2)Y(3)-0.0313Y(0)Y(1)X(2)X(3)-0.0313X(0)X(1)Y(2)Y(3)+0.0313X(0)Y(1)Y(2)X(3)+0.0824Z(0)Z(2)+0.1137Z(0)Z(3)+0.0437Y(0)Z(1)Y(2)Z(3)+0.0437X(0)Z(1)X(2)Z(3)+0.1137Z(1)Z(2)+0.0437Y(1)Y(3)+0.0437X(1)X(3)+0.0824Z(1)Z(3)+0.1209Z(2)Z(3)
finished after 61.363147020339966s

molecule: BeH2, distance: 0.5
-11.3512+0.1932Z(0)-0.0200Y(0)Z(1)Y(2)-0.0200X(0)Z(1)X(2)+0.1932Z(1)-0.0200Y(1)Z(2)Y(3)-0.0200X(1)Z(2)X(3)+0.1932Z(2)+0.1932Z(3)-0.3459Z(4)-0.2074Y(4)Z(5)Y(6)-0.2074X(4)Z(5)X(6)-0.3459Z(5)-0.2074Y(5)Z(6)Y(7)-0.2074X(5)Z(6)X(7)-0.3459Z(6)-0.3459Z(7)+0.1720Z(0)Z(1)+0.0134Y(0)Y(2)+0.0134X(0)X(2)+0.0134Z(0)Y(1)Z(2)Y(3)+0.0134Z(0)X(1)Z(2)X(3)+0.0045Y(0)X(1)X(2)Y(3)-0.0045Y(0)Y(1)X(2)X(3)-0.0045X(0)X(1)Y(2)Y(3)+0.0045X(0)Y(1)Y(2)X(3)+0.0351Y(0)X(1)X(4)Y(5)-0.0351Y(0)Y(1)X(4)X(5)-0.0351X(0)X(1)Y(4)Y(5)+0.0351X(0)Y(1)Y(4)X(5)+0.0001Y(0)X(1)X(5)Y(6)+0.0001Y(0)Y(1)Y(5)Y(6)+0.0001X(0)X(1)X(5)X(6)+0.0001X(0)Y(1)Y(5)X(6)-0.0001Y(0)X(1)X(4)Z(5)Z(6)Y(7)+0.0001Y(0)Y(1)X(4)Z(5)Z(6)X(7)+0.0001X(0)X(1)Y(4)Z(5)Z(6)Y(7)-0.0001X(0)Y(1)Y(4)Z(5)Z(6)X(7)+0.0008Y(0)X(1)X(6)Y(7)-0.0008Y(0)Y(1)X(6)X(7)-0.0008X(0)X(1)Y(6)Y(7)+0.0008X(0)Y(1)Y(6)X(7)+0.0953Z(0)Z(2)+0.0080Y(0)Z(1)X(2)X(4)Z(5)Y(6)+0.0002Y(0)Z(1)Y(2)Y(4)Z(5)Y(6)-0.0079Y(0)Z(1)Y(2)X(4)Z(5)X(6)-0.0079X(0)Z(1)X(2)Y(4)Z(5)Y(6)+0.0002X(0)Z(1)X(2)X(4)Z(5)X(6)+0.0080X(0)Z(1)Y(2)Y(4)Z(5)X(6)+0.0998Z(0)Z(3)+0.0134Y(0)Z(1)Y(2)Z(3)+0.0134X(0)Z(1)X(2)Z(3)+0.0003Y(0)Z(1)Z(2)X(3)X(4)Y(5)-0.0003Y(0)Z(1)Z(2)Y(3)X(4)X(5)-0.0003X(0)Z(1)Z(2)X(3)Y(4)Y(5)+0.0003X(0)Z(1)Z(2)Y(3)Y(4)X(5)-0.0006Y(0)Z(1)Z(2)X(3)X(5)Y(6)-0.0006Y(0)Z(1)Z(2)Y(3)Y(5)Y(6)-0.0006X(0)Z(1)Z(2)X(3)X(5)X(6)-0.0006X(0)Z(1)Z(2)Y(3)Y(5)X(6)+0.0086Y(0)Z(1)Z(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0086Y(0)Z(1)Z(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0086X(0)Z(1)Z(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0086X(0)Z(1)Z(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0003Y(0)Z(1)Z(2)X(3)X(6)Y(7)-0.0003Y(0)Z(1)Z(2)Y(3)X(6)X(7)-0.0003X(0)Z(1)Z(2)X(3)Y(6)Y(7)+0.0003X(0)Z(1)Z(2)Y(3)Y(6)X(7)+0.1313Z(0)Z(4)+0.0011Z(0)Y(4)Z(5)Y(6)+0.0011Z(0)X(4)Z(5)X(6)+0.0108Y(0)Z(1)Y(2)Z(4)+0.0108X(0)Z(1)X(2)Z(4)+0.1664Z(0)Z(5)+0.0105Y(0)Z(1)Y(2)Z(5)+0.0105X(0)Z(1)X(2)Z(5)+0.0012Z(0)Y(5)Z(6)Y(7)+0.0012Z(0)X(5)Z(6)X(7)+0.0007Y(0)Z(1)Y(2)Y(5)Z(6)Y(7)+0.0007Y(0)Z(1)Y(2)X(5)Z(6)X(7)+0.0007X(0)Z(1)X(2)Y(5)Z(6)Y(7)+0.0007X(0)Z(1)X(2)X(5)Z(6)X(7)+0.0982Z(0)Z(6)+0.0108Y(0)Z(1)Y(2)Z(6)+0.0108X(0)Z(1)X(2)Z(6)+0.0991Z(0)Z(7)+0.0105Y(0)Z(1)Y(2)Z(7)+0.0105X(0)Z(1)X(2)Z(7)+0.0998Z(1)Z(2)+0.0134Y(1)Y(3)+0.0134X(1)X(3)-0.0003Y(1)X(2)X(4)Y(5)-0.0003Y(1)Y(2)Y(4)Y(5)-0.0003X(1)X(2)X(4)X(5)-0.0003X(1)Y(2)Y(4)X(5)-0.0006Y(1)X(2)X(4)Z(5)Z(6)Y(7)-0.0006Y(1)Y(2)Y(4)Z(5)Z(6)Y(7)-0.0006X(1)X(2)X(4)Z(5)Z(6)X(7)-0.0006X(1)Y(2)Y(4)Z(5)Z(6)X(7)+0.0086Y(1)X(2)X(5)Y(6)-0.0086Y(1)Y(2)X(5)X(6)-0.0086X(1)X(2)Y(5)Y(6)+0.0086X(1)Y(2)Y(5)X(6)-0.0003Y(1)X(2)X(6)Y(7)-0.0003Y(1)Y(2)Y(6)Y(7)-0.0003X(1)X(2)X(6)X(7)-0.0003X(1)Y(2)Y(6)X(7)+0.0953Z(1)Z(3)+0.0080Y(1)Z(2)X(3)X(5)Z(6)Y(7)+0.0002Y(1)Z(2)Y(3)Y(5)Z(6)Y(7)-0.0079Y(1)Z(2)Y(3)X(5)Z(6)X(7)-0.0079X(1)Z(2)X(3)Y(5)Z(6)Y(7)+0.0002X(1)Z(2)X(3)X(5)Z(6)X(7)+0.0080X(1)Z(2)Y(3)Y(5)Z(6)X(7)+0.1664Z(1)Z(4)+0.0105Y(1)Z(2)Y(3)Z(4)+0.0105X(1)Z(2)X(3)Z(4)+0.0012Z(1)Y(4)Z(5)Y(6)+0.0012Z(1)X(4)Z(5)X(6)+0.0007Y(1)Z(2)Y(3)Y(4)Z(5)Y(6)+0.0007Y(1)Z(2)Y(3)X(4)Z(5)X(6)+0.0007X(1)Z(2)X(3)Y(4)Z(5)Y(6)+0.0007X(1)Z(2)X(3)X(4)Z(5)X(6)+0.1313Z(1)Z(5)+0.0011Z(1)Y(5)Z(6)Y(7)+0.0011Z(1)X(5)Z(6)X(7)+0.0108Y(1)Z(2)Y(3)Z(5)+0.0108X(1)Z(2)X(3)Z(5)+0.0991Z(1)Z(6)+0.0105Y(1)Z(2)Y(3)Z(6)+0.0105X(1)Z(2)X(3)Z(6)+0.0982Z(1)Z(7)+0.0108Y(1)Z(2)Y(3)Z(7)+0.0108X(1)Z(2)X(3)Z(7)+0.1720Z(2)Z(3)+0.0008Y(2)X(3)X(4)Y(5)-0.0008Y(2)Y(3)X(4)X(5)-0.0008X(2)X(3)Y(4)Y(5)+0.0008X(2)Y(3)Y(4)X(5)+0.0001Y(2)X(3)X(5)Y(6)+0.0001Y(2)Y(3)Y(5)Y(6)+0.0001X(2)X(3)X(5)X(6)+0.0001X(2)Y(3)Y(5)X(6)-0.0001Y(2)X(3)X(4)Z(5)Z(6)Y(7)+0.0001Y(2)Y(3)X(4)Z(5)Z(6)X(7)+0.0001X(2)X(3)Y(4)Z(5)Z(6)Y(7)-0.0001X(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0351Y(2)X(3)X(6)Y(7)-0.0351Y(2)Y(3)X(6)X(7)-0.0351X(2)X(3)Y(6)Y(7)+0.0351X(2)Y(3)Y(6)X(7)+0.0982Z(2)Z(4)+0.0011Z(2)Y(4)Z(5)Y(6)+0.0011Z(2)X(4)Z(5)X(6)+0.0991Z(2)Z(5)+0.0012Z(2)Y(5)Z(6)Y(7)+0.0012Z(2)X(5)Z(6)X(7)+0.1313Z(2)Z(6)+0.1664Z(2)Z(7)+0.0991Z(3)Z(4)+0.0012Z(3)Y(4)Z(5)Y(6)+0.0012Z(3)X(4)Z(5)X(6)+0.0982Z(3)Z(5)+0.0011Z(3)Y(5)Z(6)Y(7)+0.0011Z(3)X(5)Z(6)X(7)+0.1664Z(3)Z(6)+0.1313Z(3)Z(7)+0.1787Z(4)Z(5)+0.0005Y(4)Y(6)+0.0005X(4)X(6)+0.0005Z(4)Y(5)Z(6)Y(7)+0.0005Z(4)X(5)Z(6)X(7)+0.0011Y(4)X(5)X(6)Y(7)-0.0011Y(4)Y(5)X(6)X(7)-0.0011X(4)X(5)Y(6)Y(7)+0.0011X(4)Y(5)Y(6)X(7)+0.1002Z(4)Z(6)+0.1013Z(4)Z(7)+0.0005Y(4)Z(5)Y(6)Z(7)+0.0005X(4)Z(5)X(6)Z(7)+0.1013Z(5)Z(6)+0.0005Y(5)Y(7)+0.0005X(5)X(7)+0.1002Z(5)Z(7)+0.1787Z(6)Z(7)
finished after 17.732813358306885s

molecule: H2, distance: 0.735
-0.0613+0.1709Z(0)+0.1709Z(1)-0.2480Z(2)-0.2480Z(3)+0.1652Z(0)Z(1)+0.0435Y(0)X(1)X(2)Y(3)-0.0435Y(0)Y(1)X(2)X(3)-0.0435X(0)X(1)Y(2)Y(3)+0.0435X(0)Y(1)Y(2)X(3)+0.1245Z(0)Z(2)+0.1680Z(0)Z(3)+0.1680Z(1)Z(2)+0.1245Z(1)Z(3)+0.1853Z(2)Z(3)
finished after 37.74236798286438s

molecule: LiH, distance: 0.735
-6.9006+0.0846Z(0)+0.0054Y(0)Z(1)Y(2)+0.0054X(0)Z(1)X(2)+0.0846Z(1)+0.0054Y(1)Z(2)Y(3)+0.0054X(1)Z(2)X(3)-0.2130Z(2)-0.2130Z(3)+0.1357Z(0)Z(1)-0.0428Y(0)Y(2)-0.0428X(0)X(2)-0.0428Z(0)Y(1)Z(2)Y(3)-0.0428Z(0)X(1)Z(2)X(3)+0.0363Y(0)X(1)X(2)Y(3)-0.0363Y(0)Y(1)X(2)X(3)-0.0363X(0)X(1)Y(2)Y(3)+0.0363X(0)Y(1)Y(2)X(3)+0.0933Z(0)Z(2)+0.1296Z(0)Z(3)-0.0482Y(0)Z(1)Y(2)Z(3)-0.0482X(0)Z(1)X(2)Z(3)+0.1296Z(1)Z(2)-0.0482Y(1)Y(3)-0.0482X(1)X(3)+0.0933Z(1)Z(3)+0.1332Z(2)Z(3)
finished after 66.95733165740967s

molecule: BeH2, distance: 0.735
-12.4834+0.1836Z(0)-0.0107Y(0)Z(1)Y(2)-0.0107X(0)Z(1)X(2)+0.0106Y(0)Z(1)Z(2)Z(3)Y(4)+0.0106X(0)Z(1)Z(2)Z(3)X(4)+0.0035Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)+0.0035X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)+0.1836Z(1)-0.0107Y(1)Z(2)Y(3)-0.0107X(1)Z(2)X(3)+0.0106Y(1)Z(2)Z(3)Z(4)Y(5)+0.0106X(1)Z(2)Z(3)Z(4)X(5)+0.0035Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0035X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.1836Z(2)+0.0035Y(2)Z(3)Y(4)+0.0035X(2)Z(3)X(4)+0.0106Y(2)Z(3)Z(4)Z(5)Y(6)+0.0106X(2)Z(3)Z(4)Z(5)X(6)+0.1836Z(3)+0.0035Y(3)Z(4)Y(5)+0.0035X(3)Z(4)X(5)+0.0106Y(3)Z(4)Z(5)Z(6)Y(7)+0.0106X(3)Z(4)Z(5)Z(6)X(7)-0.3892Z(4)-0.0247Y(4)Z(5)Y(6)-0.0247X(4)Z(5)X(6)-0.3892Z(5)-0.0247Y(5)Z(6)Y(7)-0.0247X(5)Z(6)X(7)-0.3892Z(6)-0.3892Z(7)+0.1701Z(0)Z(1)+0.0069Y(0)Y(2)+0.0069X(0)X(2)-0.0110Y(0)Z(2)Z(3)Y(4)-0.0110X(0)Z(2)Z(3)X(4)-0.0489Y(0)Z(2)Z(3)Z(4)Z(5)Y(6)-0.0489X(0)Z(2)Z(3)Z(4)Z(5)X(6)+0.0069Z(0)Y(1)Z(2)Y(3)+0.0069Z(0)X(1)Z(2)X(3)+0.0015Y(0)X(1)X(2)Y(3)-0.0015Y(0)Y(1)X(2)X(3)-0.0015X(0)X(1)Y(2)Y(3)+0.0015X(0)Y(1)Y(2)X(3)+0.0001Y(0)X(1)X(3)Y(4)+0.0001Y(0)Y(1)Y(3)Y(4)+0.0001X(0)X(1)X(3)X(4)+0.0001X(0)Y(1)Y(3)X(4)+0.0051Y(0)X(1)X(3)Z(4)Z(5)Y(6)+0.0051Y(0)Y(1)Y(3)Z(4)Z(5)Y(6)+0.0051X(0)X(1)X(3)Z(4)Z(5)X(6)+0.0051X(0)Y(1)Y(3)Z(4)Z(5)X(6)-0.0110Z(0)Y(1)Z(2)Z(3)Z(4)Y(5)-0.0110Z(0)X(1)Z(2)Z(3)Z(4)X(5)-0.0001Y(0)X(1)X(2)Z(3)Z(4)Y(5)+0.0001Y(0)Y(1)X(2)Z(3)Z(4)X(5)+0.0001X(0)X(1)Y(2)Z(3)Z(4)Y(5)-0.0001X(0)Y(1)Y(2)Z(3)Z(4)X(5)+0.0034Y(0)X(1)X(4)Y(5)-0.0034Y(0)Y(1)X(4)X(5)-0.0034X(0)X(1)Y(4)Y(5)+0.0034X(0)Y(1)Y(4)X(5)-0.0043Y(0)X(1)X(5)Y(6)-0.0043Y(0)Y(1)Y(5)Y(6)-0.0043X(0)X(1)X(5)X(6)-0.0043X(0)Y(1)Y(5)X(6)-0.0489Z(0)Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0489Z(0)X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)-0.0051Y(0)X(1)X(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0051Y(0)Y(1)X(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.0051X(0)X(1)Y(2)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0051X(0)Y(1)Y(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.0043Y(0)X(1)X(4)Z(5)Z(6)Y(7)-0.0043Y(0)Y(1)X(4)Z(5)Z(6)X(7)-0.0043X(0)X(1)Y(4)Z(5)Z(6)Y(7)+0.0043X(0)Y(1)Y(4)Z(5)Z(6)X(7)+0.0408Y(0)X(1)X(6)Y(7)-0.0408Y(0)Y(1)X(6)X(7)-0.0408X(0)X(1)Y(6)Y(7)+0.0408X(0)Y(1)Y(6)X(7)+0.0828Z(0)Z(2)-0.0097Z(0)Y(2)Z(3)Y(4)-0.0097Z(0)X(2)Z(3)X(4)+0.0056Z(0)Y(2)Z(3)Z(4)Z(5)Y(6)+0.0056Z(0)X(2)Z(3)Z(4)Z(5)X(6)+0.0056Y(0)Z(1)Z(3)Y(4)+0.0056X(0)Z(1)Z(3)X(4)-0.0097Y(0)Z(1)Z(3)Z(4)Z(5)Y(6)-0.0097X(0)Z(1)Z(3)Z(4)Z(5)X(6)-0.0047Y(0)Z(1)X(2)X(4)Z(5)Y(6)-0.0033Y(0)Z(1)Y(2)Y(4)Z(5)Y(6)+0.0014Y(0)Z(1)Y(2)X(4)Z(5)X(6)+0.0014X(0)Z(1)X(2)Y(4)Z(5)Y(6)-0.0033X(0)Z(1)X(2)X(4)Z(5)X(6)-0.0047X(0)Z(1)Y(2)Y(4)Z(5)X(6)+0.0842Z(0)Z(3)+0.0069Y(0)Z(1)Y(2)Z(3)+0.0069X(0)Z(1)X(2)Z(3)+0.0108Y(0)Z(1)Z(2)Y(4)+0.0108X(0)Z(1)Z(2)X(4)-0.0096Y(0)Z(1)Z(2)Z(4)Z(5)Y(6)-0.0096X(0)Z(1)Z(2)Z(4)Z(5)X(6)-0.0096Z(0)Y(3)Z(4)Y(5)-0.0096Z(0)X(3)Z(4)X(5)-0.0051Y(0)Z(1)Y(2)Y(3)Z(4)Y(5)-0.0051Y(0)Z(1)Y(2)X(3)Z(4)X(5)-0.0051X(0)Z(1)X(2)Y(3)Z(4)Y(5)-0.0051X(0)Z(1)X(2)X(3)Z(4)X(5)-0.0055Y(0)Z(1)Z(2)X(3)X(4)Y(5)+0.0055Y(0)Z(1)Z(2)Y(3)X(4)X(5)+0.0055X(0)Z(1)Z(2)X(3)Y(4)Y(5)-0.0055X(0)Z(1)Z(2)Y(3)Y(4)X(5)-0.0026Y(0)Z(1)Z(2)X(3)X(5)Y(6)-0.0026Y(0)Z(1)Z(2)Y(3)Y(5)Y(6)-0.0026X(0)Z(1)Z(2)X(3)X(5)X(6)-0.0026X(0)Z(1)Z(2)Y(3)Y(5)X(6)+0.0108Z(0)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0108Z(0)X(3)Z(4)Z(5)Z(6)X(7)-0.0001Y(0)Z(1)Y(2)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0001Y(0)Z(1)Y(2)X(3)Z(4)Z(5)Z(6)X(7)-0.0001X(0)Z(1)X(2)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0001X(0)Z(1)X(2)X(3)Z(4)Z(5)Z(6)X(7)-0.0022Y(0)Z(1)Z(2)X(3)X(4)Z(5)Z(6)Y(7)+0.0022Y(0)Z(1)Z(2)Y(3)X(4)Z(5)Z(6)X(7)+0.0022X(0)Z(1)Z(2)X(3)Y(4)Z(5)Z(6)Y(7)-0.0022X(0)Z(1)Z(2)Y(3)Y(4)Z(5)Z(6)X(7)-0.0055Y(0)Z(1)Z(2)X(3)X(6)Y(7)+0.0055Y(0)Z(1)Z(2)Y(3)X(6)X(7)+0.0055X(0)Z(1)Z(2)X(3)Y(6)Y(7)-0.0055X(0)Z(1)Z(2)Y(3)Y(6)X(7)+0.0765Z(0)Z(4)-0.0020Z(0)Y(4)Z(5)Y(6)-0.0020Z(0)X(4)Z(5)X(6)+0.0018Y(0)Z(1)Y(2)Z(4)+0.0018X(0)Z(1)X(2)Z(4)-0.0088Y(0)Z(1)Z(2)Z(3)Z(5)Y(6)-0.0088X(0)Z(1)Z(2)Z(3)Z(5)X(6)+0.0799Z(0)Z(5)+0.0074Y(0)Z(1)Y(2)Z(5)+0.0074X(0)Z(1)X(2)Z(5)+0.0110Y(0)Z(1)Z(2)Z(3)Y(4)Z(5)+0.0110X(0)Z(1)Z(2)Z(3)X(4)Z(5)-0.0083Y(0)Z(1)Z(2)Z(3)Z(4)Y(6)-0.0083X(0)Z(1)Z(2)Z(3)Z(4)X(6)-0.0063Z(0)Y(5)Z(6)Y(7)-0.0063Z(0)X(5)Z(6)X(7)-0.0008Y(0)Z(1)Y(2)Y(5)Z(6)Y(7)-0.0008Y(0)Z(1)Y(2)X(5)Z(6)X(7)-0.0008X(0)Z(1)X(2)Y(5)Z(6)Y(7)-0.0008X(0)Z(1)X(2)X(5)Z(6)X(7)-0.0004Y(0)Z(1)Z(2)Z(3)Y(4)Y(5)Z(6)Y(7)-0.0004Y(0)Z(1)Z(2)Z(3)Y(4)X(5)Z(6)X(7)-0.0004X(0)Z(1)Z(2)Z(3)X(4)Y(5)Z(6)Y(7)-0.0004X(0)Z(1)Z(2)Z(3)X(4)X(5)Z(6)X(7)+0.0045Y(0)Z(1)Z(2)Z(3)Z(4)X(5)X(6)Y(7)-0.0045Y(0)Z(1)Z(2)Z(3)Z(4)Y(5)X(6)X(7)-0.0045X(0)Z(1)Z(2)Z(3)Z(4)X(5)Y(6)Y(7)+0.0045X(0)Z(1)Z(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.1209Z(0)Z(6)+0.0018Y(0)Z(1)Y(2)Z(6)+0.0018X(0)Z(1)X(2)Z(6)-0.0057Y(0)Z(1)Z(2)Z(3)Y(4)Z(6)-0.0057X(0)Z(1)Z(2)Z(3)X(4)Z(6)+0.1616Z(0)Z(7)+0.0074Y(0)Z(1)Y(2)Z(7)+0.0074X(0)Z(1)X(2)Z(7)-0.0103Y(0)Z(1)Z(2)Z(3)Y(4)Z(7)-0.0103X(0)Z(1)Z(2)Z(3)X(4)Z(7)-0.0546Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)Z(7)-0.0546X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.0842Z(1)Z(2)+0.0069Y(1)Y(3)+0.0069X(1)X(3)+0.0108Y(1)Z(3)Z(4)Y(5)+0.0108X(1)Z(3)Z(4)X(5)-0.0096Y(1)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0096X(1)Z(3)Z(4)Z(5)Z(6)X(7)-0.0096Z(1)Y(2)Z(3)Y(4)-0.0096Z(1)X(2)Z(3)X(4)-0.0051Y(1)X(2)X(3)Y(4)+0.0051Y(1)Y(2)X(3)X(4)+0.0051X(1)X(2)Y(3)Y(4)-0.0051X(1)Y(2)Y(3)X(4)+0.0055Y(1)X(2)X(4)Y(5)+0.0055Y(1)Y(2)Y(4)Y(5)+0.0055X(1)X(2)X(4)X(5)+0.0055X(1)Y(2)Y(4)X(5)-0.0026Y(1)X(2)X(4)Z(5)Z(6)Y(7)-0.0026Y(1)Y(2)Y(4)Z(5)Z(6)Y(7)-0.0026X(1)X(2)X(4)Z(5)Z(6)X(7)-0.0026X(1)Y(2)Y(4)Z(5)Z(6)X(7)+0.0108Z(1)Y(2)Z(3)Z(4)Z(5)Y(6)+0.0108Z(1)X(2)Z(3)Z(4)Z(5)X(6)-0.0001Y(1)X(2)X(3)Z(4)Z(5)Y(6)+0.0001Y(1)Y(2)X(3)Z(4)Z(5)X(6)+0.0001X(1)X(2)Y(3)Z(4)Z(5)Y(6)-0.0001X(1)Y(2)Y(3)Z(4)Z(5)X(6)-0.0022Y(1)X(2)X(5)Y(6)+0.0022Y(1)Y(2)X(5)X(6)+0.0022X(1)X(2)Y(5)Y(6)-0.0022X(1)Y(2)Y(5)X(6)+0.0055Y(1)X(2)X(6)Y(7)+0.0055Y(1)Y(2)Y(6)Y(7)+0.0055X(1)X(2)X(6)X(7)+0.0055X(1)Y(2)Y(6)X(7)+0.0828Z(1)Z(3)-0.0097Z(1)Y(3)Z(4)Y(5)-0.0097Z(1)X(3)Z(4)X(5)+0.0056Z(1)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0056Z(1)X(3)Z(4)Z(5)Z(6)X(7)+0.0056Y(1)Z(2)Z(4)Y(5)+0.0056X(1)Z(2)Z(4)X(5)-0.0097Y(1)Z(2)Z(4)Z(5)Z(6)Y(7)-0.0097X(1)Z(2)Z(4)Z(5)Z(6)X(7)-0.0047Y(1)Z(2)X(3)X(5)Z(6)Y(7)-0.0033Y(1)Z(2)Y(3)Y(5)Z(6)Y(7)+0.0014Y(1)Z(2)Y(3)X(5)Z(6)X(7)+0.0014X(1)Z(2)X(3)Y(5)Z(6)Y(7)-0.0033X(1)Z(2)X(3)X(5)Z(6)X(7)-0.0047X(1)Z(2)Y(3)Y(5)Z(6)X(7)+0.0799Z(1)Z(4)+0.0074Y(1)Z(2)Y(3)Z(4)+0.0074X(1)Z(2)X(3)Z(4)+0.0110Y(1)Z(2)Z(3)Y(5)+0.0110X(1)Z(2)Z(3)X(5)-0.0083Y(1)Z(2)Z(3)Z(5)Z(6)Y(7)-0.0083X(1)Z(2)Z(3)Z(5)Z(6)X(7)-0.0063Z(1)Y(4)Z(5)Y(6)-0.0063Z(1)X(4)Z(5)X(6)-0.0008Y(1)Z(2)Y(3)Y(4)Z(5)Y(6)-0.0008Y(1)Z(2)Y(3)X(4)Z(5)X(6)-0.0008X(1)Z(2)X(3)Y(4)Z(5)Y(6)-0.0008X(1)Z(2)X(3)X(4)Z(5)X(6)-0.0004Y(1)Z(2)Z(3)X(4)X(5)Y(6)+0.0004Y(1)Z(2)Z(3)Y(4)X(5)X(6)+0.0004X(1)Z(2)Z(3)X(4)Y(5)Y(6)-0.0004X(1)Z(2)Z(3)Y(4)Y(5)X(6)-0.0045Y(1)Z(2)Z(3)X(4)X(6)Y(7)-0.0045Y(1)Z(2)Z(3)Y(4)Y(6)Y(7)-0.0045X(1)Z(2)Z(3)X(4)X(6)X(7)-0.0045X(1)Z(2)Z(3)Y(4)Y(6)X(7)+0.0765Z(1)Z(5)-0.0020Z(1)Y(5)Z(6)Y(7)-0.0020Z(1)X(5)Z(6)X(7)+0.0018Y(1)Z(2)Y(3)Z(5)+0.0018X(1)Z(2)X(3)Z(5)-0.0088Y(1)Z(2)Z(3)Z(4)Z(6)Y(7)-0.0088X(1)Z(2)Z(3)Z(4)Z(6)X(7)+0.1616Z(1)Z(6)+0.0074Y(1)Z(2)Y(3)Z(6)+0.0074X(1)Z(2)X(3)Z(6)-0.0103Y(1)Z(2)Z(3)Z(4)Y(5)Z(6)-0.0103X(1)Z(2)Z(3)Z(4)X(5)Z(6)-0.0546Y(1)Z(2)Z(3)Z(4)Z(5)Y(7)-0.0546X(1)Z(2)Z(3)Z(4)Z(5)X(7)+0.1209Z(1)Z(7)+0.0018Y(1)Z(2)Y(3)Z(7)+0.0018X(1)Z(2)X(3)Z(7)-0.0057Y(1)Z(2)Z(3)Z(4)Y(5)Z(7)-0.0057X(1)Z(2)Z(3)Z(4)X(5)Z(7)+0.1701Z(2)Z(3)-0.0489Y(2)Y(4)-0.0489X(2)X(4)-0.0110Y(2)Z(4)Z(5)Y(6)-0.0110X(2)Z(4)Z(5)X(6)-0.0489Z(2)Y(3)Z(4)Y(5)-0.0489Z(2)X(3)Z(4)X(5)+0.0408Y(2)X(3)X(4)Y(5)-0.0408Y(2)Y(3)X(4)X(5)-0.0408X(2)X(3)Y(4)Y(5)+0.0408X(2)Y(3)Y(4)X(5)-0.0043Y(2)X(3)X(5)Y(6)-0.0043Y(2)Y(3)Y(5)Y(6)-0.0043X(2)X(3)X(5)X(6)-0.0043X(2)Y(3)Y(5)X(6)-0.0110Z(2)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0110Z(2)X(3)Z(4)Z(5)Z(6)X(7)+0.0043Y(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0043Y(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0043X(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0043X(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0034Y(2)X(3)X(6)Y(7)-0.0034Y(2)Y(3)X(6)X(7)-0.0034X(2)X(3)Y(6)Y(7)+0.0034X(2)Y(3)Y(6)X(7)+0.1209Z(2)Z(4)-0.0020Z(2)Y(4)Z(5)Y(6)-0.0020Z(2)X(4)Z(5)X(6)-0.0057Y(2)Z(3)Z(5)Y(6)-0.0057X(2)Z(3)Z(5)X(6)+0.1616Z(2)Z(5)-0.0546Y(2)Z(3)Y(4)Z(5)-0.0546X(2)Z(3)X(4)Z(5)-0.0103Y(2)Z(3)Z(4)Y(6)-0.0103X(2)Z(3)Z(4)X(6)-0.0063Z(2)Y(5)Z(6)Y(7)-0.0063Z(2)X(5)Z(6)X(7)+0.0045Y(2)Z(3)Y(4)Y(5)Z(6)Y(7)+0.0045Y(2)Z(3)Y(4)X(5)Z(6)X(7)+0.0045X(2)Z(3)X(4)Y(5)Z(6)Y(7)+0.0045X(2)Z(3)X(4)X(5)Z(6)X(7)-0.0004Y(2)Z(3)Z(4)X(5)X(6)Y(7)+0.0004Y(2)Z(3)Z(4)Y(5)X(6)X(7)+0.0004X(2)Z(3)Z(4)X(5)Y(6)Y(7)-0.0004X(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.0765Z(2)Z(6)-0.0088Y(2)Z(3)Y(4)Z(6)-0.0088X(2)Z(3)X(4)Z(6)+0.0799Z(2)Z(7)-0.0083Y(2)Z(3)Y(4)Z(7)-0.0083X(2)Z(3)X(4)Z(7)+0.0110Y(2)Z(3)Z(4)Z(5)Y(6)Z(7)+0.0110X(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.1616Z(3)Z(4)-0.0546Y(3)Y(5)-0.0546X(3)X(5)-0.0103Y(3)Z(5)Z(6)Y(7)-0.0103X(3)Z(5)Z(6)X(7)-0.0063Z(3)Y(4)Z(5)Y(6)-0.0063Z(3)X(4)Z(5)X(6)+0.0045Y(3)X(4)X(5)Y(6)-0.0045Y(3)Y(4)X(5)X(6)-0.0045X(3)X(4)Y(5)Y(6)+0.0045X(3)Y(4)Y(5)X(6)+0.0004Y(3)X(4)X(6)Y(7)+0.0004Y(3)Y(4)Y(6)Y(7)+0.0004X(3)X(4)X(6)X(7)+0.0004X(3)Y(4)Y(6)X(7)+0.1209Z(3)Z(5)-0.0020Z(3)Y(5)Z(6)Y(7)-0.0020Z(3)X(5)Z(6)X(7)-0.0057Y(3)Z(4)Z(6)Y(7)-0.0057X(3)Z(4)Z(6)X(7)+0.0799Z(3)Z(6)-0.0083Y(3)Z(4)Y(5)Z(6)-0.0083X(3)Z(4)X(5)Z(6)+0.0110Y(3)Z(4)Z(5)Y(7)+0.0110X(3)Z(4)Z(5)X(7)+0.0765Z(3)Z(7)-0.0088Y(3)Z(4)Y(5)Z(7)-0.0088X(3)Z(4)X(5)Z(7)+0.1653Z(4)Z(5)-0.0064Y(4)Y(6)-0.0064X(4)X(6)-0.0064Z(4)Y(5)Z(6)Y(7)-0.0064Z(4)X(5)Z(6)X(7)+0.0014Y(4)X(5)X(6)Y(7)-0.0014Y(4)Y(5)X(6)X(7)-0.0014X(4)X(5)Y(6)Y(7)+0.0014X(4)Y(5)Y(6)X(7)+0.0744Z(4)Z(6)+0.0758Z(4)Z(7)-0.0064Y(4)Z(5)Y(6)Z(7)-0.0064X(4)Z(5)X(6)Z(7)+0.0758Z(5)Z(6)-0.0064Y(5)Y(7)-0.0064X(5)X(7)+0.0744Z(5)Z(7)+0.1653Z(6)Z(7)
finished after 19.099634885787964s

molecule: H2, distance: 1.0
-0.3527+0.1523Z(0)+0.1523Z(1)-0.1225Z(2)-0.1225Z(3)+0.1447Z(0)Z(1)+0.0427Y(0)X(1)X(2)Y(3)-0.0427Y(0)Y(1)X(2)X(3)-0.0427X(0)X(1)Y(2)Y(3)+0.0427X(0)Y(1)Y(2)X(3)+0.1065Z(0)Z(2)+0.1492Z(0)Z(3)+0.1492Z(1)Z(2)+0.1065Z(1)Z(3)+0.1662Z(2)Z(3)
finished after 30.72659134864807s

molecule: LiH, distance: 1.0
-7.0907+0.0826Z(0)-0.0044Y(0)Z(1)Y(2)-0.0044X(0)Z(1)X(2)+0.0826Z(1)-0.0044Y(1)Z(2)Y(3)-0.0044X(1)Z(2)X(3)-0.2274Z(2)-0.2274Z(3)+0.1356Z(0)Z(1)+0.0425Y(0)Y(2)+0.0425X(0)X(2)+0.0425Z(0)Y(1)Z(2)Y(3)+0.0425Z(0)X(1)Z(2)X(3)+0.0351Y(0)X(1)X(2)Y(3)-0.0351Y(0)Y(1)X(2)X(3)-0.0351X(0)X(1)Y(2)Y(3)+0.0351X(0)Y(1)Y(2)X(3)+0.0935Z(0)Z(2)+0.1286Z(0)Z(3)+0.0469Y(0)Z(1)Y(2)Z(3)+0.0469X(0)Z(1)X(2)Z(3)+0.1286Z(1)Z(2)+0.0469Y(1)Y(3)+0.0469X(1)X(3)+0.0935Z(1)Z(3)+0.1311Z(2)Z(3)
finished after 53.49275279045105s

molecule: BeH2, distance: 1.0
-13.1367+0.1624Z(0)-0.0121Y(0)Z(1)Y(2)-0.0121X(0)Z(1)X(2)+0.0086Y(0)Z(1)Z(2)Z(3)Y(4)+0.0086X(0)Z(1)Z(2)Z(3)X(4)+0.0054Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)+0.0054X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)+0.1624Z(1)-0.0121Y(1)Z(2)Y(3)-0.0121X(1)Z(2)X(3)+0.0086Y(1)Z(2)Z(3)Z(4)Y(5)+0.0086X(1)Z(2)Z(3)Z(4)X(5)+0.0054Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0054X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.1624Z(2)+0.0054Y(2)Z(3)Y(4)+0.0054X(2)Z(3)X(4)+0.0086Y(2)Z(3)Z(4)Z(5)Y(6)+0.0086X(2)Z(3)Z(4)Z(5)X(6)+0.1624Z(3)+0.0054Y(3)Z(4)Y(5)+0.0054X(3)Z(4)X(5)+0.0086Y(3)Z(4)Z(5)Z(6)Y(7)+0.0086X(3)Z(4)Z(5)Z(6)X(7)-0.3636Z(4)-0.0637Y(4)Z(5)Y(6)-0.0637X(4)Z(5)X(6)-0.3636Z(5)-0.0637Y(5)Z(6)Y(7)-0.0637X(5)Z(6)X(7)-0.3636Z(6)-0.3636Z(7)+0.1581Z(0)Z(1)+0.0035Y(0)Y(2)+0.0035X(0)X(2)-0.0068Y(0)Z(2)Z(3)Y(4)-0.0068X(0)Z(2)Z(3)X(4)-0.0450Y(0)Z(2)Z(3)Z(4)Z(5)Y(6)-0.0450X(0)Z(2)Z(3)Z(4)Z(5)X(6)+0.0035Z(0)Y(1)Z(2)Y(3)+0.0035Z(0)X(1)Z(2)X(3)+0.0006Y(0)X(1)X(2)Y(3)-0.0006Y(0)Y(1)X(2)X(3)-0.0006X(0)X(1)Y(2)Y(3)+0.0006X(0)Y(1)Y(2)X(3)+0.0002Y(0)X(1)X(3)Y(4)+0.0002Y(0)Y(1)Y(3)Y(4)+0.0002X(0)X(1)X(3)X(4)+0.0002X(0)Y(1)Y(3)X(4)+0.0029Y(0)X(1)X(3)Z(4)Z(5)Y(6)+0.0029Y(0)Y(1)Y(3)Z(4)Z(5)Y(6)+0.0029X(0)X(1)X(3)Z(4)Z(5)X(6)+0.0029X(0)Y(1)Y(3)Z(4)Z(5)X(6)-0.0068Z(0)Y(1)Z(2)Z(3)Z(4)Y(5)-0.0068Z(0)X(1)Z(2)Z(3)Z(4)X(5)-0.0002Y(0)X(1)X(2)Z(3)Z(4)Y(5)+0.0002Y(0)Y(1)X(2)Z(3)Z(4)X(5)+0.0002X(0)X(1)Y(2)Z(3)Z(4)Y(5)-0.0002X(0)Y(1)Y(2)Z(3)Z(4)X(5)+0.0016Y(0)X(1)X(4)Y(5)-0.0016Y(0)Y(1)X(4)X(5)-0.0016X(0)X(1)Y(4)Y(5)+0.0016X(0)Y(1)Y(4)X(5)-0.0030Y(0)X(1)X(5)Y(6)-0.0030Y(0)Y(1)Y(5)Y(6)-0.0030X(0)X(1)X(5)X(6)-0.0030X(0)Y(1)Y(5)X(6)-0.0450Z(0)Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0450Z(0)X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)-0.0029Y(0)X(1)X(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0029Y(0)Y(1)X(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.0029X(0)X(1)Y(2)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0029X(0)Y(1)Y(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.0030Y(0)X(1)X(4)Z(5)Z(6)Y(7)-0.0030Y(0)Y(1)X(4)Z(5)Z(6)X(7)-0.0030X(0)X(1)Y(4)Z(5)Z(6)Y(7)+0.0030X(0)Y(1)Y(4)Z(5)Z(6)X(7)+0.0370Y(0)X(1)X(6)Y(7)-0.0370Y(0)Y(1)X(6)X(7)-0.0370X(0)X(1)Y(6)Y(7)+0.0370X(0)Y(1)Y(6)X(7)+0.0704Z(0)Z(2)-0.0043Z(0)Y(2)Z(3)Y(4)-0.0043Z(0)X(2)Z(3)X(4)+0.0045Z(0)Y(2)Z(3)Z(4)Z(5)Y(6)+0.0045Z(0)X(2)Z(3)Z(4)Z(5)X(6)+0.0045Y(0)Z(1)Z(3)Y(4)+0.0045X(0)Z(1)Z(3)X(4)-0.0043Y(0)Z(1)Z(3)Z(4)Z(5)Y(6)-0.0043X(0)Z(1)Z(3)Z(4)Z(5)X(6)-0.0014Y(0)Z(1)X(2)X(4)Z(5)Y(6)-0.0007Y(0)Z(1)Y(2)Y(4)Z(5)Y(6)+0.0007Y(0)Z(1)Y(2)X(4)Z(5)X(6)+0.0007X(0)Z(1)X(2)Y(4)Z(5)Y(6)-0.0007X(0)Z(1)X(2)X(4)Z(5)X(6)-0.0014X(0)Z(1)Y(2)Y(4)Z(5)X(6)+0.0710Z(0)Z(3)+0.0035Y(0)Z(1)Y(2)Z(3)+0.0035X(0)Z(1)X(2)Z(3)+0.0074Y(0)Z(1)Z(2)Y(4)+0.0074X(0)Z(1)Z(2)X(4)-0.0041Y(0)Z(1)Z(2)Z(4)Z(5)Y(6)-0.0041X(0)Z(1)Z(2)Z(4)Z(5)X(6)-0.0041Z(0)Y(3)Z(4)Y(5)-0.0041Z(0)X(3)Z(4)X(5)-0.0029Y(0)Z(1)Y(2)Y(3)Z(4)Y(5)-0.0029Y(0)Z(1)Y(2)X(3)Z(4)X(5)-0.0029X(0)Z(1)X(2)Y(3)Z(4)Y(5)-0.0029X(0)Z(1)X(2)X(3)Z(4)X(5)-0.0032Y(0)Z(1)Z(2)X(3)X(4)Y(5)+0.0032Y(0)Z(1)Z(2)Y(3)X(4)X(5)+0.0032X(0)Z(1)Z(2)X(3)Y(4)Y(5)-0.0032X(0)Z(1)Z(2)Y(3)Y(4)X(5)-0.0006Y(0)Z(1)Z(2)X(3)X(5)Y(6)-0.0006Y(0)Z(1)Z(2)Y(3)Y(5)Y(6)-0.0006X(0)Z(1)Z(2)X(3)X(5)X(6)-0.0006X(0)Z(1)Z(2)Y(3)Y(5)X(6)+0.0074Z(0)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0074Z(0)X(3)Z(4)Z(5)Z(6)X(7)-0.0002Y(0)Z(1)Y(2)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0002Y(0)Z(1)Y(2)X(3)Z(4)Z(5)Z(6)X(7)-0.0002X(0)Z(1)X(2)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0002X(0)Z(1)X(2)X(3)Z(4)Z(5)Z(6)X(7)-0.0008Y(0)Z(1)Z(2)X(3)X(4)Z(5)Z(6)Y(7)+0.0008Y(0)Z(1)Z(2)Y(3)X(4)Z(5)Z(6)X(7)+0.0008X(0)Z(1)Z(2)X(3)Y(4)Z(5)Z(6)Y(7)-0.0008X(0)Z(1)Z(2)Y(3)Y(4)Z(5)Z(6)X(7)-0.0032Y(0)Z(1)Z(2)X(3)X(6)Y(7)+0.0032Y(0)Z(1)Z(2)Y(3)X(6)X(7)+0.0032X(0)Z(1)Z(2)X(3)Y(6)Y(7)-0.0032X(0)Z(1)Z(2)Y(3)Y(6)X(7)+0.0652Z(0)Z(4)-0.0009Z(0)Y(4)Z(5)Y(6)-0.0009Z(0)X(4)Z(5)X(6)+0.0006Y(0)Z(1)Y(2)Z(4)+0.0006X(0)Z(1)X(2)Z(4)-0.0037Y(0)Z(1)Z(2)Z(3)Z(5)Y(6)-0.0037X(0)Z(1)Z(2)Z(3)Z(5)X(6)+0.0668Z(0)Z(5)+0.0038Y(0)Z(1)Y(2)Z(5)+0.0038X(0)Z(1)X(2)Z(5)+0.0074Y(0)Z(1)Z(2)Z(3)Y(4)Z(5)+0.0074X(0)Z(1)Z(2)Z(3)X(4)Z(5)-0.0033Y(0)Z(1)Z(2)Z(3)Z(4)Y(6)-0.0033X(0)Z(1)Z(2)Z(3)Z(4)X(6)-0.0040Z(0)Y(5)Z(6)Y(7)-0.0040Z(0)X(5)Z(6)X(7)-0.0001Y(0)Z(1)Y(2)Y(5)Z(6)Y(7)-0.0001Y(0)Z(1)Y(2)X(5)Z(6)X(7)-0.0001X(0)Z(1)X(2)Y(5)Z(6)Y(7)-0.0001X(0)Z(1)X(2)X(5)Z(6)X(7)-0.0004Y(0)Z(1)Z(2)Z(3)Y(4)Y(5)Z(6)Y(7)-0.0004Y(0)Z(1)Z(2)Z(3)Y(4)X(5)Z(6)X(7)-0.0004X(0)Z(1)Z(2)Z(3)X(4)Y(5)Z(6)Y(7)-0.0004X(0)Z(1)Z(2)Z(3)X(4)X(5)Z(6)X(7)+0.0020Y(0)Z(1)Z(2)Z(3)Z(4)X(5)X(6)Y(7)-0.0020Y(0)Z(1)Z(2)Z(3)Z(4)Y(5)X(6)X(7)-0.0020X(0)Z(1)Z(2)Z(3)Z(4)X(5)Y(6)Y(7)+0.0020X(0)Z(1)Z(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.1145Z(0)Z(6)+0.0006Y(0)Z(1)Y(2)Z(6)+0.0006X(0)Z(1)X(2)Z(6)-0.0044Y(0)Z(1)Z(2)Z(3)Y(4)Z(6)-0.0044X(0)Z(1)Z(2)Z(3)X(4)Z(6)+0.1515Z(0)Z(7)+0.0038Y(0)Z(1)Y(2)Z(7)+0.0038X(0)Z(1)X(2)Z(7)-0.0064Y(0)Z(1)Z(2)Z(3)Y(4)Z(7)-0.0064X(0)Z(1)Z(2)Z(3)X(4)Z(7)-0.0518Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)Z(7)-0.0518X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.0710Z(1)Z(2)+0.0035Y(1)Y(3)+0.0035X(1)X(3)+0.0074Y(1)Z(3)Z(4)Y(5)+0.0074X(1)Z(3)Z(4)X(5)-0.0041Y(1)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0041X(1)Z(3)Z(4)Z(5)Z(6)X(7)-0.0041Z(1)Y(2)Z(3)Y(4)-0.0041Z(1)X(2)Z(3)X(4)-0.0029Y(1)X(2)X(3)Y(4)+0.0029Y(1)Y(2)X(3)X(4)+0.0029X(1)X(2)Y(3)Y(4)-0.0029X(1)Y(2)Y(3)X(4)+0.0032Y(1)X(2)X(4)Y(5)+0.0032Y(1)Y(2)Y(4)Y(5)+0.0032X(1)X(2)X(4)X(5)+0.0032X(1)Y(2)Y(4)X(5)-0.0006Y(1)X(2)X(4)Z(5)Z(6)Y(7)-0.0006Y(1)Y(2)Y(4)Z(5)Z(6)Y(7)-0.0006X(1)X(2)X(4)Z(5)Z(6)X(7)-0.0006X(1)Y(2)Y(4)Z(5)Z(6)X(7)+0.0074Z(1)Y(2)Z(3)Z(4)Z(5)Y(6)+0.0074Z(1)X(2)Z(3)Z(4)Z(5)X(6)-0.0002Y(1)X(2)X(3)Z(4)Z(5)Y(6)+0.0002Y(1)Y(2)X(3)Z(4)Z(5)X(6)+0.0002X(1)X(2)Y(3)Z(4)Z(5)Y(6)-0.0002X(1)Y(2)Y(3)Z(4)Z(5)X(6)-0.0008Y(1)X(2)X(5)Y(6)+0.0008Y(1)Y(2)X(5)X(6)+0.0008X(1)X(2)Y(5)Y(6)-0.0008X(1)Y(2)Y(5)X(6)+0.0032Y(1)X(2)X(6)Y(7)+0.0032Y(1)Y(2)Y(6)Y(7)+0.0032X(1)X(2)X(6)X(7)+0.0032X(1)Y(2)Y(6)X(7)+0.0704Z(1)Z(3)-0.0043Z(1)Y(3)Z(4)Y(5)-0.0043Z(1)X(3)Z(4)X(5)+0.0045Z(1)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0045Z(1)X(3)Z(4)Z(5)Z(6)X(7)+0.0045Y(1)Z(2)Z(4)Y(5)+0.0045X(1)Z(2)Z(4)X(5)-0.0043Y(1)Z(2)Z(4)Z(5)Z(6)Y(7)-0.0043X(1)Z(2)Z(4)Z(5)Z(6)X(7)-0.0014Y(1)Z(2)X(3)X(5)Z(6)Y(7)-0.0007Y(1)Z(2)Y(3)Y(5)Z(6)Y(7)+0.0007Y(1)Z(2)Y(3)X(5)Z(6)X(7)+0.0007X(1)Z(2)X(3)Y(5)Z(6)Y(7)-0.0007X(1)Z(2)X(3)X(5)Z(6)X(7)-0.0014X(1)Z(2)Y(3)Y(5)Z(6)X(7)+0.0668Z(1)Z(4)+0.0038Y(1)Z(2)Y(3)Z(4)+0.0038X(1)Z(2)X(3)Z(4)+0.0074Y(1)Z(2)Z(3)Y(5)+0.0074X(1)Z(2)Z(3)X(5)-0.0033Y(1)Z(2)Z(3)Z(5)Z(6)Y(7)-0.0033X(1)Z(2)Z(3)Z(5)Z(6)X(7)-0.0040Z(1)Y(4)Z(5)Y(6)-0.0040Z(1)X(4)Z(5)X(6)-0.0001Y(1)Z(2)Y(3)Y(4)Z(5)Y(6)-0.0001Y(1)Z(2)Y(3)X(4)Z(5)X(6)-0.0001X(1)Z(2)X(3)Y(4)Z(5)Y(6)-0.0001X(1)Z(2)X(3)X(4)Z(5)X(6)-0.0004Y(1)Z(2)Z(3)X(4)X(5)Y(6)+0.0004Y(1)Z(2)Z(3)Y(4)X(5)X(6)+0.0004X(1)Z(2)Z(3)X(4)Y(5)Y(6)-0.0004X(1)Z(2)Z(3)Y(4)Y(5)X(6)-0.0020Y(1)Z(2)Z(3)X(4)X(6)Y(7)-0.0020Y(1)Z(2)Z(3)Y(4)Y(6)Y(7)-0.0020X(1)Z(2)Z(3)X(4)X(6)X(7)-0.0020X(1)Z(2)Z(3)Y(4)Y(6)X(7)+0.0652Z(1)Z(5)-0.0009Z(1)Y(5)Z(6)Y(7)-0.0009Z(1)X(5)Z(6)X(7)+0.0006Y(1)Z(2)Y(3)Z(5)+0.0006X(1)Z(2)X(3)Z(5)-0.0037Y(1)Z(2)Z(3)Z(4)Z(6)Y(7)-0.0037X(1)Z(2)Z(3)Z(4)Z(6)X(7)+0.1515Z(1)Z(6)+0.0038Y(1)Z(2)Y(3)Z(6)+0.0038X(1)Z(2)X(3)Z(6)-0.0064Y(1)Z(2)Z(3)Z(4)Y(5)Z(6)-0.0064X(1)Z(2)Z(3)Z(4)X(5)Z(6)-0.0518Y(1)Z(2)Z(3)Z(4)Z(5)Y(7)-0.0518X(1)Z(2)Z(3)Z(4)Z(5)X(7)+0.1145Z(1)Z(7)+0.0006Y(1)Z(2)Y(3)Z(7)+0.0006X(1)Z(2)X(3)Z(7)-0.0044Y(1)Z(2)Z(3)Z(4)Y(5)Z(7)-0.0044X(1)Z(2)Z(3)Z(4)X(5)Z(7)+0.1581Z(2)Z(3)-0.0450Y(2)Y(4)-0.0450X(2)X(4)-0.0068Y(2)Z(4)Z(5)Y(6)-0.0068X(2)Z(4)Z(5)X(6)-0.0450Z(2)Y(3)Z(4)Y(5)-0.0450Z(2)X(3)Z(4)X(5)+0.0370Y(2)X(3)X(4)Y(5)-0.0370Y(2)Y(3)X(4)X(5)-0.0370X(2)X(3)Y(4)Y(5)+0.0370X(2)Y(3)Y(4)X(5)-0.0030Y(2)X(3)X(5)Y(6)-0.0030Y(2)Y(3)Y(5)Y(6)-0.0030X(2)X(3)X(5)X(6)-0.0030X(2)Y(3)Y(5)X(6)-0.0068Z(2)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0068Z(2)X(3)Z(4)Z(5)Z(6)X(7)+0.0030Y(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0030Y(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0030X(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0030X(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0016Y(2)X(3)X(6)Y(7)-0.0016Y(2)Y(3)X(6)X(7)-0.0016X(2)X(3)Y(6)Y(7)+0.0016X(2)Y(3)Y(6)X(7)+0.1145Z(2)Z(4)-0.0009Z(2)Y(4)Z(5)Y(6)-0.0009Z(2)X(4)Z(5)X(6)-0.0044Y(2)Z(3)Z(5)Y(6)-0.0044X(2)Z(3)Z(5)X(6)+0.1515Z(2)Z(5)-0.0518Y(2)Z(3)Y(4)Z(5)-0.0518X(2)Z(3)X(4)Z(5)-0.0064Y(2)Z(3)Z(4)Y(6)-0.0064X(2)Z(3)Z(4)X(6)-0.0040Z(2)Y(5)Z(6)Y(7)-0.0040Z(2)X(5)Z(6)X(7)+0.0020Y(2)Z(3)Y(4)Y(5)Z(6)Y(7)+0.0020Y(2)Z(3)Y(4)X(5)Z(6)X(7)+0.0020X(2)Z(3)X(4)Y(5)Z(6)Y(7)+0.0020X(2)Z(3)X(4)X(5)Z(6)X(7)-0.0004Y(2)Z(3)Z(4)X(5)X(6)Y(7)+0.0004Y(2)Z(3)Z(4)Y(5)X(6)X(7)+0.0004X(2)Z(3)Z(4)X(5)Y(6)Y(7)-0.0004X(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.0652Z(2)Z(6)-0.0037Y(2)Z(3)Y(4)Z(6)-0.0037X(2)Z(3)X(4)Z(6)+0.0668Z(2)Z(7)-0.0033Y(2)Z(3)Y(4)Z(7)-0.0033X(2)Z(3)X(4)Z(7)+0.0074Y(2)Z(3)Z(4)Z(5)Y(6)Z(7)+0.0074X(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.1515Z(3)Z(4)-0.0518Y(3)Y(5)-0.0518X(3)X(5)-0.0064Y(3)Z(5)Z(6)Y(7)-0.0064X(3)Z(5)Z(6)X(7)-0.0040Z(3)Y(4)Z(5)Y(6)-0.0040Z(3)X(4)Z(5)X(6)+0.0020Y(3)X(4)X(5)Y(6)-0.0020Y(3)Y(4)X(5)X(6)-0.0020X(3)X(4)Y(5)Y(6)+0.0020X(3)Y(4)Y(5)X(6)+0.0004Y(3)X(4)X(6)Y(7)+0.0004Y(3)Y(4)Y(6)Y(7)+0.0004X(3)X(4)X(6)X(7)+0.0004X(3)Y(4)Y(6)X(7)+0.1145Z(3)Z(5)-0.0009Z(3)Y(5)Z(6)Y(7)-0.0009Z(3)X(5)Z(6)X(7)-0.0044Y(3)Z(4)Z(6)Y(7)-0.0044X(3)Z(4)Z(6)X(7)+0.0668Z(3)Z(6)-0.0033Y(3)Z(4)Y(5)Z(6)-0.0033X(3)Z(4)X(5)Z(6)+0.0074Y(3)Z(4)Z(5)Y(7)+0.0074X(3)Z(4)Z(5)X(7)+0.0652Z(3)Z(7)-0.0037Y(3)Z(4)Y(5)Z(7)-0.0037X(3)Z(4)X(5)Z(7)+0.1571Z(4)Z(5)-0.0037Y(4)Y(6)-0.0037X(4)X(6)-0.0037Z(4)Y(5)Z(6)Y(7)-0.0037Z(4)X(5)Z(6)X(7)+0.0008Y(4)X(5)X(6)Y(7)-0.0008Y(4)Y(5)X(6)X(7)-0.0008X(4)X(5)Y(6)Y(7)+0.0008X(4)Y(5)Y(6)X(7)+0.0626Z(4)Z(6)+0.0634Z(4)Z(7)-0.0037Y(4)Z(5)Y(6)Z(7)-0.0037X(4)Z(5)X(6)Z(7)+0.0634Z(5)Z(6)-0.0037Y(5)Y(7)-0.0037X(5)X(7)+0.0626Z(5)Z(7)+0.1571Z(6)Z(7)
finished after 20.003359079360962s

molecule: H2, distance: 1.5
-0.5759+0.1321Z(0)+0.1321Z(1)-0.0051Z(2)-0.0051Z(3)+0.1191Z(0)Z(1)+0.0426Y(0)X(1)X(2)Y(3)-0.0426Y(0)Y(1)X(2)X(3)-0.0426X(0)X(1)Y(2)Y(3)+0.0426X(0)Y(1)Y(2)X(3)+0.0824Z(0)Z(2)+0.1250Z(0)Z(3)+0.1250Z(1)Z(2)+0.0824Z(1)Z(3)+0.1409Z(2)Z(3)
finished after 34.57813882827759s

molecule: LiH, distance: 1.5
-7.2700+0.0716Z(0)+0.0066Y(0)Z(1)Y(2)+0.0066X(0)Z(1)X(2)+0.0716Z(1)+0.0066Y(1)Z(2)Y(3)+0.0066X(1)Z(2)X(3)-0.2044Z(2)-0.2044Z(3)+0.1229Z(0)Z(1)-0.0375Y(0)Y(2)-0.0375X(0)X(2)-0.0375Z(0)Y(1)Z(2)Y(3)-0.0375Z(0)X(1)Z(2)X(3)+0.0322Y(0)X(1)X(2)Y(3)-0.0322Y(0)Y(1)X(2)X(3)-0.0322X(0)X(1)Y(2)Y(3)+0.0322X(0)Y(1)Y(2)X(3)+0.0870Z(0)Z(2)+0.1193Z(0)Z(3)-0.0441Y(0)Z(1)Y(2)Z(3)-0.0441X(0)Z(1)X(2)Z(3)+0.1193Z(1)Z(2)-0.0441Y(1)Y(3)-0.0441X(1)X(3)+0.0870Z(1)Z(3)+0.1260Z(2)Z(3)
finished after 59.18380331993103s

molecule: BeH2, distance: 1.5
-14.0334+0.1100Z(0)-0.0127Y(0)Z(1)Y(2)-0.0127X(0)Z(1)X(2)+0.0065Y(0)Z(1)Z(2)Z(3)Y(4)+0.0065X(0)Z(1)Z(2)Z(3)X(4)+0.0059Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)+0.0059X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)+0.1100Z(1)-0.0127Y(1)Z(2)Y(3)-0.0127X(1)Z(2)X(3)+0.0065Y(1)Z(2)Z(3)Z(4)Y(5)+0.0065X(1)Z(2)Z(3)Z(4)X(5)+0.0059Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0059X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.1100Z(2)+0.0059Y(2)Z(3)Y(4)+0.0059X(2)Z(3)X(4)+0.0065Y(2)Z(3)Z(4)Z(5)Y(6)+0.0065X(2)Z(3)Z(4)Z(5)X(6)+0.1100Z(3)+0.0059Y(3)Z(4)Y(5)+0.0059X(3)Z(4)X(5)+0.0065Y(3)Z(4)Z(5)Z(6)Y(7)+0.0065X(3)Z(4)Z(5)Z(6)X(7)-0.2254Z(4)-0.0694Y(4)Z(5)Y(6)-0.0694X(4)Z(5)X(6)-0.2254Z(5)-0.0694Y(5)Z(6)Y(7)-0.0694X(5)Z(6)X(7)-0.2254Z(6)-0.2254Z(7)+0.1332Z(0)Z(1)+0.0015Y(0)Y(2)+0.0015X(0)X(2)-0.0214Y(0)Z(2)Z(3)Y(4)-0.0214X(0)Z(2)Z(3)X(4)-0.0065Y(0)Z(2)Z(3)Z(4)Z(5)Y(6)-0.0065X(0)Z(2)Z(3)Z(4)Z(5)X(6)+0.0015Z(0)Y(1)Z(2)Y(3)+0.0015Z(0)X(1)Z(2)X(3)+0.0004Y(0)X(1)X(2)Y(3)-0.0004Y(0)Y(1)X(2)X(3)-0.0004X(0)X(1)Y(2)Y(3)+0.0004X(0)Y(1)Y(2)X(3)+0.0023Y(0)X(1)X(3)Y(4)+0.0023Y(0)Y(1)Y(3)Y(4)+0.0023X(0)X(1)X(3)X(4)+0.0023X(0)Y(1)Y(3)X(4)+0.0002Y(0)X(1)X(3)Z(4)Z(5)Y(6)+0.0002Y(0)Y(1)Y(3)Z(4)Z(5)Y(6)+0.0002X(0)X(1)X(3)Z(4)Z(5)X(6)+0.0002X(0)Y(1)Y(3)Z(4)Z(5)X(6)-0.0214Z(0)Y(1)Z(2)Z(3)Z(4)Y(5)-0.0214Z(0)X(1)Z(2)Z(3)Z(4)X(5)-0.0023Y(0)X(1)X(2)Z(3)Z(4)Y(5)+0.0023Y(0)Y(1)X(2)Z(3)Z(4)X(5)+0.0023X(0)X(1)Y(2)Z(3)Z(4)Y(5)-0.0023X(0)Y(1)Y(2)Z(3)Z(4)X(5)+0.0347Y(0)X(1)X(4)Y(5)-0.0347Y(0)Y(1)X(4)X(5)-0.0347X(0)X(1)Y(4)Y(5)+0.0347X(0)Y(1)Y(4)X(5)-0.0016Y(0)X(1)X(5)Y(6)-0.0016Y(0)Y(1)Y(5)Y(6)-0.0016X(0)X(1)X(5)X(6)-0.0016X(0)Y(1)Y(5)X(6)-0.0065Z(0)Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0065Z(0)X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)-0.0002Y(0)X(1)X(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0002Y(0)Y(1)X(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.0002X(0)X(1)Y(2)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0002X(0)Y(1)Y(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.0016Y(0)X(1)X(4)Z(5)Z(6)Y(7)-0.0016Y(0)Y(1)X(4)Z(5)Z(6)X(7)-0.0016X(0)X(1)Y(4)Z(5)Z(6)Y(7)+0.0016X(0)Y(1)Y(4)Z(5)Z(6)X(7)+0.0011Y(0)X(1)X(6)Y(7)-0.0011Y(0)Y(1)X(6)X(7)-0.0011X(0)X(1)Y(6)Y(7)+0.0011X(0)Y(1)Y(6)X(7)+0.0543Z(0)Z(2)+0.0022Z(0)Y(2)Z(3)Y(4)+0.0022Z(0)X(2)Z(3)X(4)+0.0086Z(0)Y(2)Z(3)Z(4)Z(5)Y(6)+0.0086Z(0)X(2)Z(3)Z(4)Z(5)X(6)+0.0086Y(0)Z(1)Z(3)Y(4)+0.0086X(0)Z(1)Z(3)X(4)+0.0022Y(0)Z(1)Z(3)Z(4)Z(5)Y(6)+0.0022X(0)Z(1)Z(3)Z(4)Z(5)X(6)+0.0027Y(0)Z(1)X(2)X(4)Z(5)Y(6)+0.0006Y(0)Z(1)Y(2)Y(4)Z(5)Y(6)-0.0021Y(0)Z(1)Y(2)X(4)Z(5)X(6)-0.0021X(0)Z(1)X(2)Y(4)Z(5)Y(6)+0.0006X(0)Z(1)X(2)X(4)Z(5)X(6)+0.0027X(0)Z(1)Y(2)Y(4)Z(5)X(6)+0.0547Z(0)Z(3)+0.0015Y(0)Z(1)Y(2)Z(3)+0.0015X(0)Z(1)X(2)Z(3)+0.0088Y(0)Z(1)Z(2)Y(4)+0.0088X(0)Z(1)Z(2)X(4)+0.0045Y(0)Z(1)Z(2)Z(4)Z(5)Y(6)+0.0045X(0)Z(1)Z(2)Z(4)Z(5)X(6)+0.0045Z(0)Y(3)Z(4)Y(5)+0.0045Z(0)X(3)Z(4)X(5)-0.0002Y(0)Z(1)Y(2)Y(3)Z(4)Y(5)-0.0002Y(0)Z(1)Y(2)X(3)Z(4)X(5)-0.0002X(0)Z(1)X(2)Y(3)Z(4)Y(5)-0.0002X(0)Z(1)X(2)X(3)Z(4)X(5)-0.0018Y(0)Z(1)Z(2)X(3)X(4)Y(5)+0.0018Y(0)Z(1)Z(2)Y(3)X(4)X(5)+0.0018X(0)Z(1)Z(2)X(3)Y(4)Y(5)-0.0018X(0)Z(1)Z(2)Y(3)Y(4)X(5)+0.0004Y(0)Z(1)Z(2)X(3)X(5)Y(6)+0.0004Y(0)Z(1)Z(2)Y(3)Y(5)Y(6)+0.0004X(0)Z(1)Z(2)X(3)X(5)X(6)+0.0004X(0)Z(1)Z(2)Y(3)Y(5)X(6)+0.0088Z(0)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0088Z(0)X(3)Z(4)Z(5)Z(6)X(7)-0.0023Y(0)Z(1)Y(2)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0023Y(0)Z(1)Y(2)X(3)Z(4)Z(5)Z(6)X(7)-0.0023X(0)Z(1)X(2)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0023X(0)Z(1)X(2)X(3)Z(4)Z(5)Z(6)X(7)+0.0023Y(0)Z(1)Z(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0023Y(0)Z(1)Z(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0023X(0)Z(1)Z(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0023X(0)Z(1)Z(2)Y(3)Y(4)Z(5)Z(6)X(7)-0.0018Y(0)Z(1)Z(2)X(3)X(6)Y(7)+0.0018Y(0)Z(1)Z(2)Y(3)X(6)X(7)+0.0018X(0)Z(1)Z(2)X(3)Y(6)Y(7)-0.0018X(0)Z(1)Z(2)Y(3)Y(6)X(7)+0.1030Z(0)Z(4)+0.0033Z(0)Y(4)Z(5)Y(6)+0.0033Z(0)X(4)Z(5)X(6)-0.0002Y(0)Z(1)Y(2)Z(4)-0.0002X(0)Z(1)X(2)Z(4)-0.0048Y(0)Z(1)Z(2)Z(3)Z(5)Y(6)-0.0048X(0)Z(1)Z(2)Z(3)Z(5)X(6)+0.1378Z(0)Z(5)+0.0017Y(0)Z(1)Y(2)Z(5)+0.0017X(0)Z(1)X(2)Z(5)-0.0300Y(0)Z(1)Z(2)Z(3)Y(4)Z(5)-0.0300X(0)Z(1)Z(2)Z(3)X(4)Z(5)-0.0057Y(0)Z(1)Z(2)Z(3)Z(4)Y(6)-0.0057X(0)Z(1)Z(2)Z(3)Z(4)X(6)+0.0017Z(0)Y(5)Z(6)Y(7)+0.0017Z(0)X(5)Z(6)X(7)+0.0002Y(0)Z(1)Y(2)Y(5)Z(6)Y(7)+0.0002Y(0)Z(1)Y(2)X(5)Z(6)X(7)+0.0002X(0)Z(1)X(2)Y(5)Z(6)Y(7)+0.0002X(0)Z(1)X(2)X(5)Z(6)X(7)+0.0009Y(0)Z(1)Z(2)Z(3)Y(4)Y(5)Z(6)Y(7)+0.0009Y(0)Z(1)Z(2)Z(3)Y(4)X(5)Z(6)X(7)+0.0009X(0)Z(1)Z(2)Z(3)X(4)Y(5)Z(6)Y(7)+0.0009X(0)Z(1)Z(2)Z(3)X(4)X(5)Z(6)X(7)-0.0004Y(0)Z(1)Z(2)Z(3)Z(4)X(5)X(6)Y(7)+0.0004Y(0)Z(1)Z(2)Z(3)Z(4)Y(5)X(6)X(7)+0.0004X(0)Z(1)Z(2)Z(3)Z(4)X(5)Y(6)Y(7)-0.0004X(0)Z(1)Z(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.0581Z(0)Z(6)-0.0002Y(0)Z(1)Y(2)Z(6)-0.0002X(0)Z(1)X(2)Z(6)+0.0096Y(0)Z(1)Z(2)Z(3)Y(4)Z(6)+0.0096X(0)Z(1)Z(2)Z(3)X(4)Z(6)+0.0593Z(0)Z(7)+0.0017Y(0)Z(1)Y(2)Z(7)+0.0017X(0)Z(1)X(2)Z(7)+0.0100Y(0)Z(1)Z(2)Z(3)Y(4)Z(7)+0.0100X(0)Z(1)Z(2)Z(3)X(4)Z(7)+0.0049Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)Z(7)+0.0049X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.0547Z(1)Z(2)+0.0015Y(1)Y(3)+0.0015X(1)X(3)+0.0088Y(1)Z(3)Z(4)Y(5)+0.0088X(1)Z(3)Z(4)X(5)+0.0045Y(1)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0045X(1)Z(3)Z(4)Z(5)Z(6)X(7)+0.0045Z(1)Y(2)Z(3)Y(4)+0.0045Z(1)X(2)Z(3)X(4)-0.0002Y(1)X(2)X(3)Y(4)+0.0002Y(1)Y(2)X(3)X(4)+0.0002X(1)X(2)Y(3)Y(4)-0.0002X(1)Y(2)Y(3)X(4)+0.0018Y(1)X(2)X(4)Y(5)+0.0018Y(1)Y(2)Y(4)Y(5)+0.0018X(1)X(2)X(4)X(5)+0.0018X(1)Y(2)Y(4)X(5)+0.0004Y(1)X(2)X(4)Z(5)Z(6)Y(7)+0.0004Y(1)Y(2)Y(4)Z(5)Z(6)Y(7)+0.0004X(1)X(2)X(4)Z(5)Z(6)X(7)+0.0004X(1)Y(2)Y(4)Z(5)Z(6)X(7)+0.0088Z(1)Y(2)Z(3)Z(4)Z(5)Y(6)+0.0088Z(1)X(2)Z(3)Z(4)Z(5)X(6)-0.0023Y(1)X(2)X(3)Z(4)Z(5)Y(6)+0.0023Y(1)Y(2)X(3)Z(4)Z(5)X(6)+0.0023X(1)X(2)Y(3)Z(4)Z(5)Y(6)-0.0023X(1)Y(2)Y(3)Z(4)Z(5)X(6)+0.0023Y(1)X(2)X(5)Y(6)-0.0023Y(1)Y(2)X(5)X(6)-0.0023X(1)X(2)Y(5)Y(6)+0.0023X(1)Y(2)Y(5)X(6)+0.0018Y(1)X(2)X(6)Y(7)+0.0018Y(1)Y(2)Y(6)Y(7)+0.0018X(1)X(2)X(6)X(7)+0.0018X(1)Y(2)Y(6)X(7)+0.0543Z(1)Z(3)+0.0022Z(1)Y(3)Z(4)Y(5)+0.0022Z(1)X(3)Z(4)X(5)+0.0086Z(1)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0086Z(1)X(3)Z(4)Z(5)Z(6)X(7)+0.0086Y(1)Z(2)Z(4)Y(5)+0.0086X(1)Z(2)Z(4)X(5)+0.0022Y(1)Z(2)Z(4)Z(5)Z(6)Y(7)+0.0022X(1)Z(2)Z(4)Z(5)Z(6)X(7)+0.0027Y(1)Z(2)X(3)X(5)Z(6)Y(7)+0.0006Y(1)Z(2)Y(3)Y(5)Z(6)Y(7)-0.0021Y(1)Z(2)Y(3)X(5)Z(6)X(7)-0.0021X(1)Z(2)X(3)Y(5)Z(6)Y(7)+0.0006X(1)Z(2)X(3)X(5)Z(6)X(7)+0.0027X(1)Z(2)Y(3)Y(5)Z(6)X(7)+0.1378Z(1)Z(4)+0.0017Y(1)Z(2)Y(3)Z(4)+0.0017X(1)Z(2)X(3)Z(4)-0.0300Y(1)Z(2)Z(3)Y(5)-0.0300X(1)Z(2)Z(3)X(5)-0.0057Y(1)Z(2)Z(3)Z(5)Z(6)Y(7)-0.0057X(1)Z(2)Z(3)Z(5)Z(6)X(7)+0.0017Z(1)Y(4)Z(5)Y(6)+0.0017Z(1)X(4)Z(5)X(6)+0.0002Y(1)Z(2)Y(3)Y(4)Z(5)Y(6)+0.0002Y(1)Z(2)Y(3)X(4)Z(5)X(6)+0.0002X(1)Z(2)X(3)Y(4)Z(5)Y(6)+0.0002X(1)Z(2)X(3)X(4)Z(5)X(6)+0.0009Y(1)Z(2)Z(3)X(4)X(5)Y(6)-0.0009Y(1)Z(2)Z(3)Y(4)X(5)X(6)-0.0009X(1)Z(2)Z(3)X(4)Y(5)Y(6)+0.0009X(1)Z(2)Z(3)Y(4)Y(5)X(6)+0.0004Y(1)Z(2)Z(3)X(4)X(6)Y(7)+0.0004Y(1)Z(2)Z(3)Y(4)Y(6)Y(7)+0.0004X(1)Z(2)Z(3)X(4)X(6)X(7)+0.0004X(1)Z(2)Z(3)Y(4)Y(6)X(7)+0.1030Z(1)Z(5)+0.0033Z(1)Y(5)Z(6)Y(7)+0.0033Z(1)X(5)Z(6)X(7)-0.0002Y(1)Z(2)Y(3)Z(5)-0.0002X(1)Z(2)X(3)Z(5)-0.0048Y(1)Z(2)Z(3)Z(4)Z(6)Y(7)-0.0048X(1)Z(2)Z(3)Z(4)Z(6)X(7)+0.0593Z(1)Z(6)+0.0017Y(1)Z(2)Y(3)Z(6)+0.0017X(1)Z(2)X(3)Z(6)+0.0100Y(1)Z(2)Z(3)Z(4)Y(5)Z(6)+0.0100X(1)Z(2)Z(3)Z(4)X(5)Z(6)+0.0049Y(1)Z(2)Z(3)Z(4)Z(5)Y(7)+0.0049X(1)Z(2)Z(3)Z(4)Z(5)X(7)+0.0581Z(1)Z(7)-0.0002Y(1)Z(2)Y(3)Z(7)-0.0002X(1)Z(2)X(3)Z(7)+0.0096Y(1)Z(2)Z(3)Z(4)Y(5)Z(7)+0.0096X(1)Z(2)Z(3)Z(4)X(5)Z(7)+0.1332Z(2)Z(3)-0.0065Y(2)Y(4)-0.0065X(2)X(4)-0.0214Y(2)Z(4)Z(5)Y(6)-0.0214X(2)Z(4)Z(5)X(6)-0.0065Z(2)Y(3)Z(4)Y(5)-0.0065Z(2)X(3)Z(4)X(5)+0.0011Y(2)X(3)X(4)Y(5)-0.0011Y(2)Y(3)X(4)X(5)-0.0011X(2)X(3)Y(4)Y(5)+0.0011X(2)Y(3)Y(4)X(5)-0.0016Y(2)X(3)X(5)Y(6)-0.0016Y(2)Y(3)Y(5)Y(6)-0.0016X(2)X(3)X(5)X(6)-0.0016X(2)Y(3)Y(5)X(6)-0.0214Z(2)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0214Z(2)X(3)Z(4)Z(5)Z(6)X(7)+0.0016Y(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0016Y(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0016X(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0016X(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0347Y(2)X(3)X(6)Y(7)-0.0347Y(2)Y(3)X(6)X(7)-0.0347X(2)X(3)Y(6)Y(7)+0.0347X(2)Y(3)Y(6)X(7)+0.0581Z(2)Z(4)+0.0033Z(2)Y(4)Z(5)Y(6)+0.0033Z(2)X(4)Z(5)X(6)+0.0096Y(2)Z(3)Z(5)Y(6)+0.0096X(2)Z(3)Z(5)X(6)+0.0593Z(2)Z(5)+0.0049Y(2)Z(3)Y(4)Z(5)+0.0049X(2)Z(3)X(4)Z(5)+0.0100Y(2)Z(3)Z(4)Y(6)+0.0100X(2)Z(3)Z(4)X(6)+0.0017Z(2)Y(5)Z(6)Y(7)+0.0017Z(2)X(5)Z(6)X(7)-0.0004Y(2)Z(3)Y(4)Y(5)Z(6)Y(7)-0.0004Y(2)Z(3)Y(4)X(5)Z(6)X(7)-0.0004X(2)Z(3)X(4)Y(5)Z(6)Y(7)-0.0004X(2)Z(3)X(4)X(5)Z(6)X(7)+0.0009Y(2)Z(3)Z(4)X(5)X(6)Y(7)-0.0009Y(2)Z(3)Z(4)Y(5)X(6)X(7)-0.0009X(2)Z(3)Z(4)X(5)Y(6)Y(7)+0.0009X(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.1030Z(2)Z(6)-0.0048Y(2)Z(3)Y(4)Z(6)-0.0048X(2)Z(3)X(4)Z(6)+0.1378Z(2)Z(7)-0.0057Y(2)Z(3)Y(4)Z(7)-0.0057X(2)Z(3)X(4)Z(7)-0.0300Y(2)Z(3)Z(4)Z(5)Y(6)Z(7)-0.0300X(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.0593Z(3)Z(4)+0.0049Y(3)Y(5)+0.0049X(3)X(5)+0.0100Y(3)Z(5)Z(6)Y(7)+0.0100X(3)Z(5)Z(6)X(7)+0.0017Z(3)Y(4)Z(5)Y(6)+0.0017Z(3)X(4)Z(5)X(6)-0.0004Y(3)X(4)X(5)Y(6)+0.0004Y(3)Y(4)X(5)X(6)+0.0004X(3)X(4)Y(5)Y(6)-0.0004X(3)Y(4)Y(5)X(6)-0.0009Y(3)X(4)X(6)Y(7)-0.0009Y(3)Y(4)Y(6)Y(7)-0.0009X(3)X(4)X(6)X(7)-0.0009X(3)Y(4)Y(6)X(7)+0.0581Z(3)Z(5)+0.0033Z(3)Y(5)Z(6)Y(7)+0.0033Z(3)X(5)Z(6)X(7)+0.0096Y(3)Z(4)Z(6)Y(7)+0.0096X(3)Z(4)Z(6)X(7)+0.1378Z(3)Z(6)-0.0057Y(3)Z(4)Y(5)Z(6)-0.0057X(3)Z(4)X(5)Z(6)-0.0300Y(3)Z(4)Z(5)Y(7)-0.0300X(3)Z(4)Z(5)X(7)+0.1030Z(3)Z(7)-0.0048Y(3)Z(4)Y(5)Z(7)-0.0048X(3)Z(4)X(5)Z(7)+0.1544Z(4)Z(5)+0.0019Y(4)Y(6)+0.0019X(4)X(6)+0.0019Z(4)Y(5)Z(6)Y(7)+0.0019Z(4)X(5)Z(6)X(7)+0.0007Y(4)X(5)X(6)Y(7)-0.0007Y(4)Y(5)X(6)X(7)-0.0007X(4)X(5)Y(6)Y(7)+0.0007X(4)Y(5)Y(6)X(7)+0.0635Z(4)Z(6)+0.0642Z(4)Z(7)+0.0019Y(4)Z(5)Y(6)Z(7)+0.0019X(4)Z(5)X(6)Z(7)+0.0642Z(5)Z(6)+0.0019Y(5)Y(7)+0.0019X(5)X(7)+0.0635Z(5)Z(7)+0.1544Z(6)Z(7)
finished after 17.84534192085266s

molecule: H2, distance: 2.0
-0.6527+0.1218Z(0)+0.1218Z(1)+0.0473Z(2)+0.0473Z(3)+0.1032Z(0)Z(1)+0.0438Y(0)X(1)X(2)Y(3)-0.0438Y(0)Y(1)X(2)X(3)-0.0438X(0)X(1)Y(2)Y(3)+0.0438X(0)Y(1)Y(2)X(3)+0.0661Z(0)Z(2)+0.1098Z(0)Z(3)+0.1098Z(1)Z(2)+0.0661Z(1)Z(3)+0.1241Z(2)Z(3)
finished after 28.55091667175293s

molecule: LiH, distance: 2.0
-7.4153+0.0633Z(0)+0.0056Y(0)Z(1)Y(2)+0.0056X(0)Z(1)X(2)+0.0633Z(1)+0.0056Y(1)Z(2)Y(3)+0.0056X(1)Z(2)X(3)-0.1425Z(2)-0.1425Z(3)+0.1114Z(0)Z(1)-0.0322Y(0)Y(2)-0.0322X(0)X(2)-0.0322Z(0)Y(1)Z(2)Y(3)-0.0322Z(0)X(1)Z(2)X(3)+0.0311Y(0)X(1)X(2)Y(3)-0.0311Y(0)Y(1)X(2)X(3)-0.0311X(0)X(1)Y(2)Y(3)+0.0311X(0)Y(1)Y(2)X(3)+0.0785Z(0)Z(2)+0.1096Z(0)Z(3)-0.0379Y(0)Z(1)Y(2)Z(3)-0.0379X(0)Z(1)X(2)Z(3)+0.1096Z(1)Z(2)-0.0379Y(1)Y(3)-0.0379X(1)X(3)+0.0785Z(1)Z(3)+0.1171Z(2)Z(3)
finished after 58.36054253578186s

molecule: BeH2, distance: 2.0
-14.5739+0.1041Z(0)-0.0110Y(0)Z(1)Y(2)-0.0110X(0)Z(1)X(2)-0.0004Y(0)Z(1)Z(2)Z(3)Y(4)-0.0004X(0)Z(1)Z(2)Z(3)X(4)-0.0054Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)-0.0054X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)+0.1041Z(1)-0.0110Y(1)Z(2)Y(3)-0.0110X(1)Z(2)X(3)-0.0004Y(1)Z(2)Z(3)Z(4)Y(5)-0.0004X(1)Z(2)Z(3)Z(4)X(5)-0.0054Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0054X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.1041Z(2)-0.0054Y(2)Z(3)Y(4)-0.0054X(2)Z(3)X(4)-0.0004Y(2)Z(3)Z(4)Z(5)Y(6)-0.0004X(2)Z(3)Z(4)Z(5)X(6)+0.1041Z(3)-0.0054Y(3)Z(4)Y(5)-0.0054X(3)Z(4)X(5)-0.0004Y(3)Z(4)Z(5)Z(6)Y(7)-0.0004X(3)Z(4)Z(5)Z(6)X(7)-0.0870Z(4)-0.0488Y(4)Z(5)Y(6)-0.0488X(4)Z(5)X(6)-0.0870Z(5)-0.0488Y(5)Z(6)Y(7)-0.0488X(5)Z(6)X(7)-0.0870Z(6)-0.0870Z(7)+0.1138Z(0)Z(1)+0.0010Y(0)Y(2)+0.0010X(0)X(2)+0.0177Y(0)Z(2)Z(3)Y(4)+0.0177X(0)Z(2)Z(3)X(4)+0.0067Y(0)Z(2)Z(3)Z(4)Z(5)Y(6)+0.0067X(0)Z(2)Z(3)Z(4)Z(5)X(6)+0.0010Z(0)Y(1)Z(2)Y(3)+0.0010Z(0)X(1)Z(2)X(3)+0.0007Y(0)X(1)X(2)Y(3)-0.0007Y(0)Y(1)X(2)X(3)-0.0007X(0)X(1)Y(2)Y(3)+0.0007X(0)Y(1)Y(2)X(3)-0.0033Y(0)X(1)X(3)Y(4)-0.0033Y(0)Y(1)Y(3)Y(4)-0.0033X(0)X(1)X(3)X(4)-0.0033X(0)Y(1)Y(3)X(4)-0.0002Y(0)X(1)X(3)Z(4)Z(5)Y(6)-0.0002Y(0)Y(1)Y(3)Z(4)Z(5)Y(6)-0.0002X(0)X(1)X(3)Z(4)Z(5)X(6)-0.0002X(0)Y(1)Y(3)Z(4)Z(5)X(6)+0.0177Z(0)Y(1)Z(2)Z(3)Z(4)Y(5)+0.0177Z(0)X(1)Z(2)Z(3)Z(4)X(5)+0.0033Y(0)X(1)X(2)Z(3)Z(4)Y(5)-0.0033Y(0)Y(1)X(2)Z(3)Z(4)X(5)-0.0033X(0)X(1)Y(2)Z(3)Z(4)Y(5)+0.0033X(0)Y(1)Y(2)Z(3)Z(4)X(5)+0.0341Y(0)X(1)X(4)Y(5)-0.0341Y(0)Y(1)X(4)X(5)-0.0341X(0)X(1)Y(4)Y(5)+0.0341X(0)Y(1)Y(4)X(5)-0.0022Y(0)X(1)X(5)Y(6)-0.0022Y(0)Y(1)Y(5)Y(6)-0.0022X(0)X(1)X(5)X(6)-0.0022X(0)Y(1)Y(5)X(6)+0.0067Z(0)Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0067Z(0)X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.0002Y(0)X(1)X(2)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0002Y(0)Y(1)X(2)Z(3)Z(4)Z(5)Z(6)X(7)-0.0002X(0)X(1)Y(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0002X(0)Y(1)Y(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.0022Y(0)X(1)X(4)Z(5)Z(6)Y(7)-0.0022Y(0)Y(1)X(4)Z(5)Z(6)X(7)-0.0022X(0)X(1)Y(4)Z(5)Z(6)Y(7)+0.0022X(0)Y(1)Y(4)Z(5)Z(6)X(7)+0.0015Y(0)X(1)X(6)Y(7)-0.0015Y(0)Y(1)X(6)X(7)-0.0015X(0)X(1)Y(6)Y(7)+0.0015X(0)Y(1)Y(6)X(7)+0.0446Z(0)Z(2)-0.0028Z(0)Y(2)Z(3)Y(4)-0.0028Z(0)X(2)Z(3)X(4)-0.0085Z(0)Y(2)Z(3)Z(4)Z(5)Y(6)-0.0085Z(0)X(2)Z(3)Z(4)Z(5)X(6)-0.0085Y(0)Z(1)Z(3)Y(4)-0.0085X(0)Z(1)Z(3)X(4)-0.0028Y(0)Z(1)Z(3)Z(4)Z(5)Y(6)-0.0028X(0)Z(1)Z(3)Z(4)Z(5)X(6)+0.0037Y(0)Z(1)X(2)X(4)Z(5)Y(6)+0.0008Y(0)Z(1)Y(2)Y(4)Z(5)Y(6)-0.0029Y(0)Z(1)Y(2)X(4)Z(5)X(6)-0.0029X(0)Z(1)X(2)Y(4)Z(5)Y(6)+0.0008X(0)Z(1)X(2)X(4)Z(5)X(6)+0.0037X(0)Z(1)Y(2)Y(4)Z(5)X(6)+0.0453Z(0)Z(3)+0.0010Y(0)Z(1)Y(2)Z(3)+0.0010X(0)Z(1)X(2)Z(3)-0.0088Y(0)Z(1)Z(2)Y(4)-0.0088X(0)Z(1)Z(2)X(4)-0.0061Y(0)Z(1)Z(2)Z(4)Z(5)Y(6)-0.0061X(0)Z(1)Z(2)Z(4)Z(5)X(6)-0.0061Z(0)Y(3)Z(4)Y(5)-0.0061Z(0)X(3)Z(4)X(5)+0.0002Y(0)Z(1)Y(2)Y(3)Z(4)Y(5)+0.0002Y(0)Z(1)Y(2)X(3)Z(4)X(5)+0.0002X(0)Z(1)X(2)Y(3)Z(4)Y(5)+0.0002X(0)Z(1)X(2)X(3)Z(4)X(5)-0.0018Y(0)Z(1)Z(2)X(3)X(4)Y(5)+0.0018Y(0)Z(1)Z(2)Y(3)X(4)X(5)+0.0018X(0)Z(1)Z(2)X(3)Y(4)Y(5)-0.0018X(0)Z(1)Z(2)Y(3)Y(4)X(5)+0.0009Y(0)Z(1)Z(2)X(3)X(5)Y(6)+0.0009Y(0)Z(1)Z(2)Y(3)Y(5)Y(6)+0.0009X(0)Z(1)Z(2)X(3)X(5)X(6)+0.0009X(0)Z(1)Z(2)Y(3)Y(5)X(6)-0.0088Z(0)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0088Z(0)X(3)Z(4)Z(5)Z(6)X(7)+0.0033Y(0)Z(1)Y(2)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0033Y(0)Z(1)Y(2)X(3)Z(4)Z(5)Z(6)X(7)+0.0033X(0)Z(1)X(2)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0033X(0)Z(1)X(2)X(3)Z(4)Z(5)Z(6)X(7)+0.0028Y(0)Z(1)Z(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0028Y(0)Z(1)Z(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0028X(0)Z(1)Z(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0028X(0)Z(1)Z(2)Y(3)Y(4)Z(5)Z(6)X(7)-0.0018Y(0)Z(1)Z(2)X(3)X(6)Y(7)+0.0018Y(0)Z(1)Z(2)Y(3)X(6)X(7)+0.0018X(0)Z(1)Z(2)X(3)Y(6)Y(7)-0.0018X(0)Z(1)Z(2)Y(3)Y(6)X(7)+0.0836Z(0)Z(4)+0.0032Z(0)Y(4)Z(5)Y(6)+0.0032Z(0)X(4)Z(5)X(6)-0.0010Y(0)Z(1)Y(2)Z(4)-0.0010X(0)Z(1)X(2)Z(4)+0.0041Y(0)Z(1)Z(2)Z(3)Z(5)Y(6)+0.0041X(0)Z(1)Z(2)Z(3)Z(5)X(6)+0.1177Z(0)Z(5)+0.0008Y(0)Z(1)Y(2)Z(5)+0.0008X(0)Z(1)X(2)Z(5)+0.0213Y(0)Z(1)Z(2)Z(3)Y(4)Z(5)+0.0213X(0)Z(1)Z(2)Z(3)X(4)Z(5)+0.0058Y(0)Z(1)Z(2)Z(3)Z(4)Y(6)+0.0058X(0)Z(1)Z(2)Z(3)Z(4)X(6)+0.0010Z(0)Y(5)Z(6)Y(7)+0.0010Z(0)X(5)Z(6)X(7)-0.0001Y(0)Z(1)Y(2)Y(5)Z(6)Y(7)-0.0001Y(0)Z(1)Y(2)X(5)Z(6)X(7)-0.0001X(0)Z(1)X(2)Y(5)Z(6)Y(7)-0.0001X(0)Z(1)X(2)X(5)Z(6)X(7)-0.0018Y(0)Z(1)Z(2)Z(3)Y(4)Y(5)Z(6)Y(7)-0.0018Y(0)Z(1)Z(2)Z(3)Y(4)X(5)Z(6)X(7)-0.0018X(0)Z(1)Z(2)Z(3)X(4)Y(5)Z(6)Y(7)-0.0018X(0)Z(1)Z(2)Z(3)X(4)X(5)Z(6)X(7)+0.0003Y(0)Z(1)Z(2)Z(3)Z(4)X(5)X(6)Y(7)-0.0003Y(0)Z(1)Z(2)Z(3)Z(4)Y(5)X(6)X(7)-0.0003X(0)Z(1)Z(2)Z(3)Z(4)X(5)Y(6)Y(7)+0.0003X(0)Z(1)Z(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.0485Z(0)Z(6)-0.0010Y(0)Z(1)Y(2)Z(6)-0.0010X(0)Z(1)X(2)Z(6)-0.0101Y(0)Z(1)Z(2)Z(3)Y(4)Z(6)-0.0101X(0)Z(1)Z(2)Z(3)X(4)Z(6)+0.0500Z(0)Z(7)+0.0008Y(0)Z(1)Y(2)Z(7)+0.0008X(0)Z(1)X(2)Z(7)-0.0104Y(0)Z(1)Z(2)Z(3)Y(4)Z(7)-0.0104X(0)Z(1)Z(2)Z(3)X(4)Z(7)-0.0066Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)Z(7)-0.0066X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.0453Z(1)Z(2)+0.0010Y(1)Y(3)+0.0010X(1)X(3)-0.0088Y(1)Z(3)Z(4)Y(5)-0.0088X(1)Z(3)Z(4)X(5)-0.0061Y(1)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0061X(1)Z(3)Z(4)Z(5)Z(6)X(7)-0.0061Z(1)Y(2)Z(3)Y(4)-0.0061Z(1)X(2)Z(3)X(4)+0.0002Y(1)X(2)X(3)Y(4)-0.0002Y(1)Y(2)X(3)X(4)-0.0002X(1)X(2)Y(3)Y(4)+0.0002X(1)Y(2)Y(3)X(4)+0.0018Y(1)X(2)X(4)Y(5)+0.0018Y(1)Y(2)Y(4)Y(5)+0.0018X(1)X(2)X(4)X(5)+0.0018X(1)Y(2)Y(4)X(5)+0.0009Y(1)X(2)X(4)Z(5)Z(6)Y(7)+0.0009Y(1)Y(2)Y(4)Z(5)Z(6)Y(7)+0.0009X(1)X(2)X(4)Z(5)Z(6)X(7)+0.0009X(1)Y(2)Y(4)Z(5)Z(6)X(7)-0.0088Z(1)Y(2)Z(3)Z(4)Z(5)Y(6)-0.0088Z(1)X(2)Z(3)Z(4)Z(5)X(6)+0.0033Y(1)X(2)X(3)Z(4)Z(5)Y(6)-0.0033Y(1)Y(2)X(3)Z(4)Z(5)X(6)-0.0033X(1)X(2)Y(3)Z(4)Z(5)Y(6)+0.0033X(1)Y(2)Y(3)Z(4)Z(5)X(6)+0.0028Y(1)X(2)X(5)Y(6)-0.0028Y(1)Y(2)X(5)X(6)-0.0028X(1)X(2)Y(5)Y(6)+0.0028X(1)Y(2)Y(5)X(6)+0.0018Y(1)X(2)X(6)Y(7)+0.0018Y(1)Y(2)Y(6)Y(7)+0.0018X(1)X(2)X(6)X(7)+0.0018X(1)Y(2)Y(6)X(7)+0.0446Z(1)Z(3)-0.0028Z(1)Y(3)Z(4)Y(5)-0.0028Z(1)X(3)Z(4)X(5)-0.0085Z(1)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0085Z(1)X(3)Z(4)Z(5)Z(6)X(7)-0.0085Y(1)Z(2)Z(4)Y(5)-0.0085X(1)Z(2)Z(4)X(5)-0.0028Y(1)Z(2)Z(4)Z(5)Z(6)Y(7)-0.0028X(1)Z(2)Z(4)Z(5)Z(6)X(7)+0.0037Y(1)Z(2)X(3)X(5)Z(6)Y(7)+0.0008Y(1)Z(2)Y(3)Y(5)Z(6)Y(7)-0.0029Y(1)Z(2)Y(3)X(5)Z(6)X(7)-0.0029X(1)Z(2)X(3)Y(5)Z(6)Y(7)+0.0008X(1)Z(2)X(3)X(5)Z(6)X(7)+0.0037X(1)Z(2)Y(3)Y(5)Z(6)X(7)+0.1177Z(1)Z(4)+0.0008Y(1)Z(2)Y(3)Z(4)+0.0008X(1)Z(2)X(3)Z(4)+0.0213Y(1)Z(2)Z(3)Y(5)+0.0213X(1)Z(2)Z(3)X(5)+0.0058Y(1)Z(2)Z(3)Z(5)Z(6)Y(7)+0.0058X(1)Z(2)Z(3)Z(5)Z(6)X(7)+0.0010Z(1)Y(4)Z(5)Y(6)+0.0010Z(1)X(4)Z(5)X(6)-0.0001Y(1)Z(2)Y(3)Y(4)Z(5)Y(6)-0.0001Y(1)Z(2)Y(3)X(4)Z(5)X(6)-0.0001X(1)Z(2)X(3)Y(4)Z(5)Y(6)-0.0001X(1)Z(2)X(3)X(4)Z(5)X(6)-0.0018Y(1)Z(2)Z(3)X(4)X(5)Y(6)+0.0018Y(1)Z(2)Z(3)Y(4)X(5)X(6)+0.0018X(1)Z(2)Z(3)X(4)Y(5)Y(6)-0.0018X(1)Z(2)Z(3)Y(4)Y(5)X(6)-0.0003Y(1)Z(2)Z(3)X(4)X(6)Y(7)-0.0003Y(1)Z(2)Z(3)Y(4)Y(6)Y(7)-0.0003X(1)Z(2)Z(3)X(4)X(6)X(7)-0.0003X(1)Z(2)Z(3)Y(4)Y(6)X(7)+0.0836Z(1)Z(5)+0.0032Z(1)Y(5)Z(6)Y(7)+0.0032Z(1)X(5)Z(6)X(7)-0.0010Y(1)Z(2)Y(3)Z(5)-0.0010X(1)Z(2)X(3)Z(5)+0.0041Y(1)Z(2)Z(3)Z(4)Z(6)Y(7)+0.0041X(1)Z(2)Z(3)Z(4)Z(6)X(7)+0.0500Z(1)Z(6)+0.0008Y(1)Z(2)Y(3)Z(6)+0.0008X(1)Z(2)X(3)Z(6)-0.0104Y(1)Z(2)Z(3)Z(4)Y(5)Z(6)-0.0104X(1)Z(2)Z(3)Z(4)X(5)Z(6)-0.0066Y(1)Z(2)Z(3)Z(4)Z(5)Y(7)-0.0066X(1)Z(2)Z(3)Z(4)Z(5)X(7)+0.0485Z(1)Z(7)-0.0010Y(1)Z(2)Y(3)Z(7)-0.0010X(1)Z(2)X(3)Z(7)-0.0101Y(1)Z(2)Z(3)Z(4)Y(5)Z(7)-0.0101X(1)Z(2)Z(3)Z(4)X(5)Z(7)+0.1138Z(2)Z(3)+0.0067Y(2)Y(4)+0.0067X(2)X(4)+0.0177Y(2)Z(4)Z(5)Y(6)+0.0177X(2)Z(4)Z(5)X(6)+0.0067Z(2)Y(3)Z(4)Y(5)+0.0067Z(2)X(3)Z(4)X(5)+0.0015Y(2)X(3)X(4)Y(5)-0.0015Y(2)Y(3)X(4)X(5)-0.0015X(2)X(3)Y(4)Y(5)+0.0015X(2)Y(3)Y(4)X(5)-0.0022Y(2)X(3)X(5)Y(6)-0.0022Y(2)Y(3)Y(5)Y(6)-0.0022X(2)X(3)X(5)X(6)-0.0022X(2)Y(3)Y(5)X(6)+0.0177Z(2)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0177Z(2)X(3)Z(4)Z(5)Z(6)X(7)+0.0022Y(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0022Y(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0022X(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0022X(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0341Y(2)X(3)X(6)Y(7)-0.0341Y(2)Y(3)X(6)X(7)-0.0341X(2)X(3)Y(6)Y(7)+0.0341X(2)Y(3)Y(6)X(7)+0.0485Z(2)Z(4)+0.0032Z(2)Y(4)Z(5)Y(6)+0.0032Z(2)X(4)Z(5)X(6)-0.0101Y(2)Z(3)Z(5)Y(6)-0.0101X(2)Z(3)Z(5)X(6)+0.0500Z(2)Z(5)-0.0066Y(2)Z(3)Y(4)Z(5)-0.0066X(2)Z(3)X(4)Z(5)-0.0104Y(2)Z(3)Z(4)Y(6)-0.0104X(2)Z(3)Z(4)X(6)+0.0010Z(2)Y(5)Z(6)Y(7)+0.0010Z(2)X(5)Z(6)X(7)+0.0003Y(2)Z(3)Y(4)Y(5)Z(6)Y(7)+0.0003Y(2)Z(3)Y(4)X(5)Z(6)X(7)+0.0003X(2)Z(3)X(4)Y(5)Z(6)Y(7)+0.0003X(2)Z(3)X(4)X(5)Z(6)X(7)-0.0018Y(2)Z(3)Z(4)X(5)X(6)Y(7)+0.0018Y(2)Z(3)Z(4)Y(5)X(6)X(7)+0.0018X(2)Z(3)Z(4)X(5)Y(6)Y(7)-0.0018X(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.0836Z(2)Z(6)+0.0041Y(2)Z(3)Y(4)Z(6)+0.0041X(2)Z(3)X(4)Z(6)+0.1177Z(2)Z(7)+0.0058Y(2)Z(3)Y(4)Z(7)+0.0058X(2)Z(3)X(4)Z(7)+0.0213Y(2)Z(3)Z(4)Z(5)Y(6)Z(7)+0.0213X(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.0500Z(3)Z(4)-0.0066Y(3)Y(5)-0.0066X(3)X(5)-0.0104Y(3)Z(5)Z(6)Y(7)-0.0104X(3)Z(5)Z(6)X(7)+0.0010Z(3)Y(4)Z(5)Y(6)+0.0010Z(3)X(4)Z(5)X(6)+0.0003Y(3)X(4)X(5)Y(6)-0.0003Y(3)Y(4)X(5)X(6)-0.0003X(3)X(4)Y(5)Y(6)+0.0003X(3)Y(4)Y(5)X(6)+0.0018Y(3)X(4)X(6)Y(7)+0.0018Y(3)Y(4)Y(6)Y(7)+0.0018X(3)X(4)X(6)X(7)+0.0018X(3)Y(4)Y(6)X(7)+0.0485Z(3)Z(5)+0.0032Z(3)Y(5)Z(6)Y(7)+0.0032Z(3)X(5)Z(6)X(7)-0.0101Y(3)Z(4)Z(6)Y(7)-0.0101X(3)Z(4)Z(6)X(7)+0.1177Z(3)Z(6)+0.0058Y(3)Z(4)Y(5)Z(6)+0.0058X(3)Z(4)X(5)Z(6)+0.0213Y(3)Z(4)Z(5)Y(7)+0.0213X(3)Z(4)Z(5)X(7)+0.0836Z(3)Z(7)+0.0041Y(3)Z(4)Y(5)Z(7)+0.0041X(3)Z(4)X(5)Z(7)+0.1308Z(4)Z(5)+0.0012Y(4)Y(6)+0.0012X(4)X(6)+0.0012Z(4)Y(5)Z(6)Y(7)+0.0012Z(4)X(5)Z(6)X(7)+0.0007Y(4)X(5)X(6)Y(7)-0.0007Y(4)Y(5)X(6)X(7)-0.0007X(4)X(5)Y(6)Y(7)+0.0007X(4)Y(5)Y(6)X(7)+0.0547Z(4)Z(6)+0.0554Z(4)Z(7)+0.0012Y(4)Z(5)Y(6)Z(7)+0.0012X(4)Z(5)X(6)Z(7)+0.0554Z(5)Z(6)+0.0012Y(5)Y(7)+0.0012X(5)X(7)+0.0547Z(5)Z(7)+0.1308Z(6)Z(7)
finished after 20.87357473373413s

molecule: H2, distance: 3.0
-0.6947+0.1133Z(0)+0.1133Z(1)+0.0871Z(2)+0.0871Z(3)+0.0867Z(0)Z(1)+0.0478Y(0)X(1)X(2)Y(3)-0.0478Y(0)Y(1)X(2)X(3)-0.0478X(0)X(1)Y(2)Y(3)+0.0478X(0)Y(1)Y(2)X(3)+0.0458Z(0)Z(2)+0.0936Z(0)Z(3)+0.0936Z(1)Z(2)+0.0458Z(1)Z(3)+0.1046Z(2)Z(3)
finished after 37.312089920043945s

molecule: LiH, distance: 3.0
-7.5790+0.0627Z(0)-0.0025Y(0)Z(1)Y(2)-0.0025X(0)Z(1)X(2)+0.0627Z(1)-0.0025Y(1)Z(2)Y(3)-0.0025X(1)Z(2)X(3)-0.0493Z(2)-0.0493Z(3)+0.0973Z(0)Z(1)-0.0306Y(0)Y(2)-0.0306X(0)X(2)-0.0306Z(0)Y(1)Z(2)Y(3)-0.0306Z(0)X(1)Z(2)X(3)+0.0297Y(0)X(1)X(2)Y(3)-0.0297Y(0)Y(1)X(2)X(3)-0.0297X(0)X(1)Y(2)Y(3)+0.0297X(0)Y(1)Y(2)X(3)+0.0611Z(0)Z(2)+0.0907Z(0)Z(3)-0.0281Y(0)Z(1)Y(2)Z(3)-0.0281X(0)Z(1)X(2)Z(3)+0.0907Z(1)Z(2)-0.0281Y(1)Y(3)-0.0281X(1)X(3)+0.0611Z(1)Z(3)+0.0920Z(2)Z(3)
finished after 76.37965893745422s

molecule: BeH2, distance: 3.0
-14.8969+0.1016Z(0)+0.0089Y(0)Z(1)Y(2)+0.0089X(0)Z(1)X(2)+0.0047Y(0)Z(1)Z(2)Z(3)Y(4)+0.0047X(0)Z(1)Z(2)Z(3)X(4)+0.0030Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)+0.0030X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)+0.1016Z(1)+0.0089Y(1)Z(2)Y(3)+0.0089X(1)Z(2)X(3)+0.0047Y(1)Z(2)Z(3)Z(4)Y(5)+0.0047X(1)Z(2)Z(3)Z(4)X(5)+0.0030Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0030X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.1016Z(2)+0.0030Y(2)Z(3)Y(4)+0.0030X(2)Z(3)X(4)+0.0047Y(2)Z(3)Z(4)Z(5)Y(6)+0.0047X(2)Z(3)Z(4)Z(5)X(6)+0.1016Z(3)+0.0030Y(3)Z(4)Y(5)+0.0030X(3)Z(4)X(5)+0.0047Y(3)Z(4)Z(5)Z(6)Y(7)+0.0047X(3)Z(4)Z(5)Z(6)X(7)+0.0174Z(4)+0.0436Y(4)Z(5)Y(6)+0.0436X(4)Z(5)X(6)+0.0174Z(5)+0.0436Y(5)Z(6)Y(7)+0.0436X(5)Z(6)X(7)+0.0174Z(6)+0.0174Z(7)+0.0865Z(0)Z(1)-0.0006Y(0)Y(2)-0.0006X(0)X(2)-0.0088Y(0)Z(2)Z(3)Y(4)-0.0088X(0)Z(2)Z(3)X(4)+0.0161Y(0)Z(2)Z(3)Z(4)Z(5)Y(6)+0.0161X(0)Z(2)Z(3)Z(4)Z(5)X(6)-0.0006Z(0)Y(1)Z(2)Y(3)-0.0006Z(0)X(1)Z(2)X(3)+0.0037Y(0)X(1)X(2)Y(3)-0.0037Y(0)Y(1)X(2)X(3)-0.0037X(0)X(1)Y(2)Y(3)+0.0037X(0)Y(1)Y(2)X(3)-0.0010Y(0)X(1)X(3)Y(4)-0.0010Y(0)Y(1)Y(3)Y(4)-0.0010X(0)X(1)X(3)X(4)-0.0010X(0)Y(1)Y(3)X(4)+0.0077Y(0)X(1)X(3)Z(4)Z(5)Y(6)+0.0077Y(0)Y(1)Y(3)Z(4)Z(5)Y(6)+0.0077X(0)X(1)X(3)Z(4)Z(5)X(6)+0.0077X(0)Y(1)Y(3)Z(4)Z(5)X(6)-0.0088Z(0)Y(1)Z(2)Z(3)Z(4)Y(5)-0.0088Z(0)X(1)Z(2)Z(3)Z(4)X(5)+0.0010Y(0)X(1)X(2)Z(3)Z(4)Y(5)-0.0010Y(0)Y(1)X(2)Z(3)Z(4)X(5)-0.0010X(0)X(1)Y(2)Z(3)Z(4)Y(5)+0.0010X(0)Y(1)Y(2)Z(3)Z(4)X(5)+0.0051Y(0)X(1)X(4)Y(5)-0.0051Y(0)Y(1)X(4)X(5)-0.0051X(0)X(1)Y(4)Y(5)+0.0051X(0)Y(1)Y(4)X(5)+0.0054Y(0)X(1)X(5)Y(6)+0.0054Y(0)Y(1)Y(5)Y(6)+0.0054X(0)X(1)X(5)X(6)+0.0054X(0)Y(1)Y(5)X(6)+0.0161Z(0)Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0161Z(0)X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)-0.0077Y(0)X(1)X(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0077Y(0)Y(1)X(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.0077X(0)X(1)Y(2)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0077X(0)Y(1)Y(2)Z(3)Z(4)Z(5)Z(6)X(7)-0.0054Y(0)X(1)X(4)Z(5)Z(6)Y(7)+0.0054Y(0)Y(1)X(4)Z(5)Z(6)X(7)+0.0054X(0)X(1)Y(4)Z(5)Z(6)Y(7)-0.0054X(0)Y(1)Y(4)Z(5)Z(6)X(7)+0.0328Y(0)X(1)X(6)Y(7)-0.0328Y(0)Y(1)X(6)X(7)-0.0328X(0)X(1)Y(6)Y(7)+0.0328X(0)Y(1)Y(6)X(7)+0.0348Z(0)Z(2)-0.0076Z(0)Y(2)Z(3)Y(4)-0.0076Z(0)X(2)Z(3)X(4)+0.0036Z(0)Y(2)Z(3)Z(4)Z(5)Y(6)+0.0036Z(0)X(2)Z(3)Z(4)Z(5)X(6)+0.0036Y(0)Z(1)Z(3)Y(4)+0.0036X(0)Z(1)Z(3)X(4)-0.0076Y(0)Z(1)Z(3)Z(4)Z(5)Y(6)-0.0076X(0)Z(1)Z(3)Z(4)Z(5)X(6)-0.0053Y(0)Z(1)X(2)X(4)Z(5)Y(6)-0.0035Y(0)Z(1)Y(2)Y(4)Z(5)Y(6)+0.0018Y(0)Z(1)Y(2)X(4)Z(5)X(6)+0.0018X(0)Z(1)X(2)Y(4)Z(5)Y(6)-0.0035X(0)Z(1)X(2)X(4)Z(5)X(6)-0.0053X(0)Z(1)Y(2)Y(4)Z(5)X(6)+0.0385Z(0)Z(3)-0.0006Y(0)Z(1)Y(2)Z(3)-0.0006X(0)Z(1)X(2)Z(3)+0.0113Y(0)Z(1)Z(2)Y(4)+0.0113X(0)Z(1)Z(2)X(4)-0.0086Y(0)Z(1)Z(2)Z(4)Z(5)Y(6)-0.0086X(0)Z(1)Z(2)Z(4)Z(5)X(6)-0.0086Z(0)Y(3)Z(4)Y(5)-0.0086Z(0)X(3)Z(4)X(5)-0.0077Y(0)Z(1)Y(2)Y(3)Z(4)Y(5)-0.0077Y(0)Z(1)Y(2)X(3)Z(4)X(5)-0.0077X(0)Z(1)X(2)Y(3)Z(4)Y(5)-0.0077X(0)Z(1)X(2)X(3)Z(4)X(5)+0.0030Y(0)Z(1)Z(2)X(3)X(4)Y(5)-0.0030Y(0)Z(1)Z(2)Y(3)X(4)X(5)-0.0030X(0)Z(1)Z(2)X(3)Y(4)Y(5)+0.0030X(0)Z(1)Z(2)Y(3)Y(4)X(5)-0.0018Y(0)Z(1)Z(2)X(3)X(5)Y(6)-0.0018Y(0)Z(1)Z(2)Y(3)Y(5)Y(6)-0.0018X(0)Z(1)Z(2)X(3)X(5)X(6)-0.0018X(0)Z(1)Z(2)Y(3)Y(5)X(6)+0.0113Z(0)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0113Z(0)X(3)Z(4)Z(5)Z(6)X(7)+0.0010Y(0)Z(1)Y(2)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0010Y(0)Z(1)Y(2)X(3)Z(4)Z(5)Z(6)X(7)+0.0010X(0)Z(1)X(2)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0010X(0)Z(1)X(2)X(3)Z(4)Z(5)Z(6)X(7)-0.0035Y(0)Z(1)Z(2)X(3)X(4)Z(5)Z(6)Y(7)+0.0035Y(0)Z(1)Z(2)Y(3)X(4)Z(5)Z(6)X(7)+0.0035X(0)Z(1)Z(2)X(3)Y(4)Z(5)Z(6)Y(7)-0.0035X(0)Z(1)Z(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0030Y(0)Z(1)Z(2)X(3)X(6)Y(7)-0.0030Y(0)Z(1)Z(2)Y(3)X(6)X(7)-0.0030X(0)Z(1)Z(2)X(3)Y(6)Y(7)+0.0030X(0)Z(1)Z(2)Y(3)Y(6)X(7)+0.0379Z(0)Z(4)-0.0051Z(0)Y(4)Z(5)Y(6)-0.0051Z(0)X(4)Z(5)X(6)+0.0033Y(0)Z(1)Y(2)Z(4)+0.0033X(0)Z(1)X(2)Z(4)-0.0094Y(0)Z(1)Z(2)Z(3)Z(5)Y(6)-0.0094X(0)Z(1)Z(2)Z(3)Z(5)X(6)+0.0430Z(0)Z(5)+0.0003Y(0)Z(1)Y(2)Z(5)+0.0003X(0)Z(1)X(2)Z(5)+0.0120Y(0)Z(1)Z(2)Z(3)Y(4)Z(5)+0.0120X(0)Z(1)Z(2)Z(3)X(4)Z(5)-0.0095Y(0)Z(1)Z(2)Z(3)Z(4)Y(6)-0.0095X(0)Z(1)Z(2)Z(3)Z(4)X(6)+0.0003Z(0)Y(5)Z(6)Y(7)+0.0003Z(0)X(5)Z(6)X(7)-0.0017Y(0)Z(1)Y(2)Y(5)Z(6)Y(7)-0.0017Y(0)Z(1)Y(2)X(5)Z(6)X(7)-0.0017X(0)Z(1)X(2)Y(5)Z(6)Y(7)-0.0017X(0)Z(1)X(2)X(5)Z(6)X(7)+0.0000Y(0)Z(1)Z(2)Z(3)Y(4)Y(5)Z(6)Y(7)+0.0000Y(0)Z(1)Z(2)Z(3)Y(4)X(5)Z(6)X(7)+0.0000X(0)Z(1)Z(2)Z(3)X(4)Y(5)Z(6)Y(7)+0.0000X(0)Z(1)Z(2)Z(3)X(4)X(5)Z(6)X(7)+0.0042Y(0)Z(1)Z(2)Z(3)Z(4)X(5)X(6)Y(7)-0.0042Y(0)Z(1)Z(2)Z(3)Z(4)Y(5)X(6)X(7)-0.0042X(0)Z(1)Z(2)Z(3)Z(4)X(5)Y(6)Y(7)+0.0042X(0)Z(1)Z(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.0563Z(0)Z(6)+0.0033Y(0)Z(1)Y(2)Z(6)+0.0033X(0)Z(1)X(2)Z(6)-0.0032Y(0)Z(1)Z(2)Z(3)Y(4)Z(6)-0.0032X(0)Z(1)Z(2)Z(3)X(4)Z(6)+0.0891Z(0)Z(7)+0.0003Y(0)Z(1)Y(2)Z(7)+0.0003X(0)Z(1)X(2)Z(7)-0.0074Y(0)Z(1)Z(2)Z(3)Y(4)Z(7)-0.0074X(0)Z(1)Z(2)Z(3)X(4)Z(7)+0.0158Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)Z(7)+0.0158X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.0385Z(1)Z(2)-0.0006Y(1)Y(3)-0.0006X(1)X(3)+0.0113Y(1)Z(3)Z(4)Y(5)+0.0113X(1)Z(3)Z(4)X(5)-0.0086Y(1)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0086X(1)Z(3)Z(4)Z(5)Z(6)X(7)-0.0086Z(1)Y(2)Z(3)Y(4)-0.0086Z(1)X(2)Z(3)X(4)-0.0077Y(1)X(2)X(3)Y(4)+0.0077Y(1)Y(2)X(3)X(4)+0.0077X(1)X(2)Y(3)Y(4)-0.0077X(1)Y(2)Y(3)X(4)-0.0030Y(1)X(2)X(4)Y(5)-0.0030Y(1)Y(2)Y(4)Y(5)-0.0030X(1)X(2)X(4)X(5)-0.0030X(1)Y(2)Y(4)X(5)-0.0018Y(1)X(2)X(4)Z(5)Z(6)Y(7)-0.0018Y(1)Y(2)Y(4)Z(5)Z(6)Y(7)-0.0018X(1)X(2)X(4)Z(5)Z(6)X(7)-0.0018X(1)Y(2)Y(4)Z(5)Z(6)X(7)+0.0113Z(1)Y(2)Z(3)Z(4)Z(5)Y(6)+0.0113Z(1)X(2)Z(3)Z(4)Z(5)X(6)+0.0010Y(1)X(2)X(3)Z(4)Z(5)Y(6)-0.0010Y(1)Y(2)X(3)Z(4)Z(5)X(6)-0.0010X(1)X(2)Y(3)Z(4)Z(5)Y(6)+0.0010X(1)Y(2)Y(3)Z(4)Z(5)X(6)-0.0035Y(1)X(2)X(5)Y(6)+0.0035Y(1)Y(2)X(5)X(6)+0.0035X(1)X(2)Y(5)Y(6)-0.0035X(1)Y(2)Y(5)X(6)-0.0030Y(1)X(2)X(6)Y(7)-0.0030Y(1)Y(2)Y(6)Y(7)-0.0030X(1)X(2)X(6)X(7)-0.0030X(1)Y(2)Y(6)X(7)+0.0348Z(1)Z(3)-0.0076Z(1)Y(3)Z(4)Y(5)-0.0076Z(1)X(3)Z(4)X(5)+0.0036Z(1)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0036Z(1)X(3)Z(4)Z(5)Z(6)X(7)+0.0036Y(1)Z(2)Z(4)Y(5)+0.0036X(1)Z(2)Z(4)X(5)-0.0076Y(1)Z(2)Z(4)Z(5)Z(6)Y(7)-0.0076X(1)Z(2)Z(4)Z(5)Z(6)X(7)-0.0053Y(1)Z(2)X(3)X(5)Z(6)Y(7)-0.0035Y(1)Z(2)Y(3)Y(5)Z(6)Y(7)+0.0018Y(1)Z(2)Y(3)X(5)Z(6)X(7)+0.0018X(1)Z(2)X(3)Y(5)Z(6)Y(7)-0.0035X(1)Z(2)X(3)X(5)Z(6)X(7)-0.0053X(1)Z(2)Y(3)Y(5)Z(6)X(7)+0.0430Z(1)Z(4)+0.0003Y(1)Z(2)Y(3)Z(4)+0.0003X(1)Z(2)X(3)Z(4)+0.0120Y(1)Z(2)Z(3)Y(5)+0.0120X(1)Z(2)Z(3)X(5)-0.0095Y(1)Z(2)Z(3)Z(5)Z(6)Y(7)-0.0095X(1)Z(2)Z(3)Z(5)Z(6)X(7)+0.0003Z(1)Y(4)Z(5)Y(6)+0.0003Z(1)X(4)Z(5)X(6)-0.0017Y(1)Z(2)Y(3)Y(4)Z(5)Y(6)-0.0017Y(1)Z(2)Y(3)X(4)Z(5)X(6)-0.0017X(1)Z(2)X(3)Y(4)Z(5)Y(6)-0.0017X(1)Z(2)X(3)X(4)Z(5)X(6)+0.0000Y(1)Z(2)Z(3)X(4)X(5)Y(6)-0.0000Y(1)Z(2)Z(3)Y(4)X(5)X(6)-0.0000X(1)Z(2)Z(3)X(4)Y(5)Y(6)+0.0000X(1)Z(2)Z(3)Y(4)Y(5)X(6)-0.0042Y(1)Z(2)Z(3)X(4)X(6)Y(7)-0.0042Y(1)Z(2)Z(3)Y(4)Y(6)Y(7)-0.0042X(1)Z(2)Z(3)X(4)X(6)X(7)-0.0042X(1)Z(2)Z(3)Y(4)Y(6)X(7)+0.0379Z(1)Z(5)-0.0051Z(1)Y(5)Z(6)Y(7)-0.0051Z(1)X(5)Z(6)X(7)+0.0033Y(1)Z(2)Y(3)Z(5)+0.0033X(1)Z(2)X(3)Z(5)-0.0094Y(1)Z(2)Z(3)Z(4)Z(6)Y(7)-0.0094X(1)Z(2)Z(3)Z(4)Z(6)X(7)+0.0891Z(1)Z(6)+0.0003Y(1)Z(2)Y(3)Z(6)+0.0003X(1)Z(2)X(3)Z(6)-0.0074Y(1)Z(2)Z(3)Z(4)Y(5)Z(6)-0.0074X(1)Z(2)Z(3)Z(4)X(5)Z(6)+0.0158Y(1)Z(2)Z(3)Z(4)Z(5)Y(7)+0.0158X(1)Z(2)Z(3)Z(4)Z(5)X(7)+0.0563Z(1)Z(7)+0.0033Y(1)Z(2)Y(3)Z(7)+0.0033X(1)Z(2)X(3)Z(7)-0.0032Y(1)Z(2)Z(3)Z(4)Y(5)Z(7)-0.0032X(1)Z(2)Z(3)Z(4)X(5)Z(7)+0.0865Z(2)Z(3)+0.0161Y(2)Y(4)+0.0161X(2)X(4)-0.0088Y(2)Z(4)Z(5)Y(6)-0.0088X(2)Z(4)Z(5)X(6)+0.0161Z(2)Y(3)Z(4)Y(5)+0.0161Z(2)X(3)Z(4)X(5)+0.0328Y(2)X(3)X(4)Y(5)-0.0328Y(2)Y(3)X(4)X(5)-0.0328X(2)X(3)Y(4)Y(5)+0.0328X(2)Y(3)Y(4)X(5)+0.0054Y(2)X(3)X(5)Y(6)+0.0054Y(2)Y(3)Y(5)Y(6)+0.0054X(2)X(3)X(5)X(6)+0.0054X(2)Y(3)Y(5)X(6)-0.0088Z(2)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0088Z(2)X(3)Z(4)Z(5)Z(6)X(7)-0.0054Y(2)X(3)X(4)Z(5)Z(6)Y(7)+0.0054Y(2)Y(3)X(4)Z(5)Z(6)X(7)+0.0054X(2)X(3)Y(4)Z(5)Z(6)Y(7)-0.0054X(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0051Y(2)X(3)X(6)Y(7)-0.0051Y(2)Y(3)X(6)X(7)-0.0051X(2)X(3)Y(6)Y(7)+0.0051X(2)Y(3)Y(6)X(7)+0.0563Z(2)Z(4)-0.0051Z(2)Y(4)Z(5)Y(6)-0.0051Z(2)X(4)Z(5)X(6)-0.0032Y(2)Z(3)Z(5)Y(6)-0.0032X(2)Z(3)Z(5)X(6)+0.0891Z(2)Z(5)+0.0158Y(2)Z(3)Y(4)Z(5)+0.0158X(2)Z(3)X(4)Z(5)-0.0074Y(2)Z(3)Z(4)Y(6)-0.0074X(2)Z(3)Z(4)X(6)+0.0003Z(2)Y(5)Z(6)Y(7)+0.0003Z(2)X(5)Z(6)X(7)+0.0042Y(2)Z(3)Y(4)Y(5)Z(6)Y(7)+0.0042Y(2)Z(3)Y(4)X(5)Z(6)X(7)+0.0042X(2)Z(3)X(4)Y(5)Z(6)Y(7)+0.0042X(2)Z(3)X(4)X(5)Z(6)X(7)+0.0000Y(2)Z(3)Z(4)X(5)X(6)Y(7)-0.0000Y(2)Z(3)Z(4)Y(5)X(6)X(7)-0.0000X(2)Z(3)Z(4)X(5)Y(6)Y(7)+0.0000X(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.0379Z(2)Z(6)-0.0094Y(2)Z(3)Y(4)Z(6)-0.0094X(2)Z(3)X(4)Z(6)+0.0430Z(2)Z(7)-0.0095Y(2)Z(3)Y(4)Z(7)-0.0095X(2)Z(3)X(4)Z(7)+0.0120Y(2)Z(3)Z(4)Z(5)Y(6)Z(7)+0.0120X(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.0891Z(3)Z(4)+0.0158Y(3)Y(5)+0.0158X(3)X(5)-0.0074Y(3)Z(5)Z(6)Y(7)-0.0074X(3)Z(5)Z(6)X(7)+0.0003Z(3)Y(4)Z(5)Y(6)+0.0003Z(3)X(4)Z(5)X(6)+0.0042Y(3)X(4)X(5)Y(6)-0.0042Y(3)Y(4)X(5)X(6)-0.0042X(3)X(4)Y(5)Y(6)+0.0042X(3)Y(4)Y(5)X(6)-0.0000Y(3)X(4)X(6)Y(7)-0.0000Y(3)Y(4)Y(6)Y(7)-0.0000X(3)X(4)X(6)X(7)-0.0000X(3)Y(4)Y(6)X(7)+0.0563Z(3)Z(5)-0.0051Z(3)Y(5)Z(6)Y(7)-0.0051Z(3)X(5)Z(6)X(7)-0.0032Y(3)Z(4)Z(6)Y(7)-0.0032X(3)Z(4)Z(6)X(7)+0.0430Z(3)Z(6)-0.0095Y(3)Z(4)Y(5)Z(6)-0.0095X(3)Z(4)X(5)Z(6)+0.0120Y(3)Z(4)Z(5)Y(7)+0.0120X(3)Z(4)Z(5)X(7)+0.0379Z(3)Z(7)-0.0094Y(3)Z(4)Y(5)Z(7)-0.0094X(3)Z(4)X(5)Z(7)+0.0972Z(4)Z(5)-0.0001Y(4)Y(6)-0.0001X(4)X(6)-0.0001Z(4)Y(5)Z(6)Y(7)-0.0001Z(4)X(5)Z(6)X(7)+0.0016Y(4)X(5)X(6)Y(7)-0.0016Y(4)Y(5)X(6)X(7)-0.0016X(4)X(5)Y(6)Y(7)+0.0016X(4)Y(5)Y(6)X(7)+0.0455Z(4)Z(6)+0.0471Z(4)Z(7)-0.0001Y(4)Z(5)Y(6)Z(7)-0.0001X(4)Z(5)X(6)Z(7)+0.0471Z(5)Z(6)-0.0001Y(5)Y(7)-0.0001X(5)X(7)+0.0455Z(5)Z(7)+0.0972Z(6)Z(7)
finished after 22.063178777694702s

molecule: H2, distance: 4.0
-0.7044+0.1110Z(0)+0.1110Z(1)+0.1001Z(2)+0.1001Z(3)+0.0799Z(0)Z(1)+0.0517Y(0)X(1)X(2)Y(3)-0.0517Y(0)Y(1)X(2)X(3)-0.0517X(0)X(1)Y(2)Y(3)+0.0517X(0)Y(1)Y(2)X(3)+0.0343Z(0)Z(2)+0.0860Z(0)Z(3)+0.0860Z(1)Z(2)+0.0343Z(1)Z(3)+0.0941Z(2)Z(3)
finished after 47.414453983306885s

molecule: LiH, distance: 4.0
-7.6533+0.0710Z(0)+0.0110Y(0)Z(1)Y(2)+0.0110X(0)Z(1)X(2)+0.0710Z(1)+0.0110Y(1)Z(2)Y(3)+0.0110X(1)Z(2)X(3)+0.0015Z(2)+0.0015Z(3)+0.0899Z(0)Z(1)+0.0317Y(0)Y(2)+0.0317X(0)X(2)+0.0317Z(0)Y(1)Z(2)Y(3)+0.0317Z(0)X(1)Z(2)X(3)+0.0279Y(0)X(1)X(2)Y(3)-0.0279Y(0)Y(1)X(2)X(3)-0.0279X(0)X(1)Y(2)Y(3)+0.0279X(0)Y(1)Y(2)X(3)+0.0468Z(0)Z(2)+0.0747Z(0)Z(3)+0.0206Y(0)Z(1)Y(2)Z(3)+0.0206X(0)Z(1)X(2)Z(3)+0.0747Z(1)Z(2)+0.0206Y(1)Y(3)+0.0206X(1)X(3)+0.0468Z(1)Z(3)+0.0715Z(2)Z(3)
finished after 96.29652762413025s

molecule: BeH2, distance: 4.0
-14.8533+0.1130Z(0)+0.0260Y(0)Z(1)Y(2)+0.0260X(0)Z(1)X(2)-0.0125Y(0)Z(1)Z(2)Z(3)Y(4)-0.0125X(0)Z(1)Z(2)Z(3)X(4)-0.0078Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)-0.0078X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)+0.1130Z(1)+0.0260Y(1)Z(2)Y(3)+0.0260X(1)Z(2)X(3)-0.0125Y(1)Z(2)Z(3)Z(4)Y(5)-0.0125X(1)Z(2)Z(3)Z(4)X(5)-0.0078Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0078X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.1130Z(2)+0.0078Y(2)Z(3)Y(4)+0.0078X(2)Z(3)X(4)+0.0125Y(2)Z(3)Z(4)Z(5)Y(6)+0.0125X(2)Z(3)Z(4)Z(5)X(6)+0.1130Z(3)+0.0078Y(3)Z(4)Y(5)+0.0078X(3)Z(4)X(5)+0.0125Y(3)Z(4)Z(5)Z(6)Y(7)+0.0125X(3)Z(4)Z(5)Z(6)X(7)+0.0214Z(4)-0.0655Y(4)Z(5)Y(6)-0.0655X(4)Z(5)X(6)+0.0214Z(5)-0.0655Y(5)Z(6)Y(7)-0.0655X(5)Z(6)X(7)+0.0214Z(6)+0.0214Z(7)+0.0660Z(0)Z(1)+0.0005Y(0)Y(2)+0.0005X(0)X(2)+0.0149Y(0)Z(2)Z(3)Y(4)+0.0149X(0)Z(2)Z(3)X(4)+0.0061Y(0)Z(2)Z(3)Z(4)Z(5)Y(6)+0.0061X(0)Z(2)Z(3)Z(4)Z(5)X(6)+0.0005Z(0)Y(1)Z(2)Y(3)+0.0005Z(0)X(1)Z(2)X(3)+0.0137Y(0)X(1)X(2)Y(3)-0.0137Y(0)Y(1)X(2)X(3)-0.0137X(0)X(1)Y(2)Y(3)+0.0137X(0)Y(1)Y(2)X(3)+0.0118Y(0)X(1)X(3)Y(4)+0.0118Y(0)Y(1)Y(3)Y(4)+0.0118X(0)X(1)X(3)X(4)+0.0118X(0)Y(1)Y(3)X(4)+0.0025Y(0)X(1)X(3)Z(4)Z(5)Y(6)+0.0025Y(0)Y(1)Y(3)Z(4)Z(5)Y(6)+0.0025X(0)X(1)X(3)Z(4)Z(5)X(6)+0.0025X(0)Y(1)Y(3)Z(4)Z(5)X(6)+0.0149Z(0)Y(1)Z(2)Z(3)Z(4)Y(5)+0.0149Z(0)X(1)Z(2)Z(3)Z(4)X(5)-0.0118Y(0)X(1)X(2)Z(3)Z(4)Y(5)+0.0118Y(0)Y(1)X(2)Z(3)Z(4)X(5)+0.0118X(0)X(1)Y(2)Z(3)Z(4)Y(5)-0.0118X(0)Y(1)Y(2)Z(3)Z(4)X(5)+0.0274Y(0)X(1)X(4)Y(5)-0.0274Y(0)Y(1)X(4)X(5)-0.0274X(0)X(1)Y(4)Y(5)+0.0274X(0)Y(1)Y(4)X(5)-0.0102Y(0)X(1)X(5)Y(6)-0.0102Y(0)Y(1)Y(5)Y(6)-0.0102X(0)X(1)X(5)X(6)-0.0102X(0)Y(1)Y(5)X(6)+0.0061Z(0)Y(1)Z(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0061Z(0)X(1)Z(2)Z(3)Z(4)Z(5)Z(6)X(7)-0.0025Y(0)X(1)X(2)Z(3)Z(4)Z(5)Z(6)Y(7)+0.0025Y(0)Y(1)X(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.0025X(0)X(1)Y(2)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0025X(0)Y(1)Y(2)Z(3)Z(4)Z(5)Z(6)X(7)+0.0102Y(0)X(1)X(4)Z(5)Z(6)Y(7)-0.0102Y(0)Y(1)X(4)Z(5)Z(6)X(7)-0.0102X(0)X(1)Y(4)Z(5)Z(6)Y(7)+0.0102X(0)Y(1)Y(4)Z(5)Z(6)X(7)+0.0121Y(0)X(1)X(6)Y(7)-0.0121Y(0)Y(1)X(6)X(7)-0.0121X(0)X(1)Y(6)Y(7)+0.0121X(0)Y(1)Y(6)X(7)+0.0301Z(0)Z(2)+0.0031Z(0)Y(2)Z(3)Y(4)+0.0031Z(0)X(2)Z(3)X(4)+0.0043Z(0)Y(2)Z(3)Z(4)Z(5)Y(6)+0.0043Z(0)X(2)Z(3)Z(4)Z(5)X(6)-0.0043Y(0)Z(1)Z(3)Y(4)-0.0043X(0)Z(1)Z(3)X(4)-0.0031Y(0)Z(1)Z(3)Z(4)Z(5)Y(6)-0.0031X(0)Z(1)Z(3)Z(4)Z(5)X(6)-0.0041Y(0)Z(1)X(2)X(4)Z(5)Y(6)-0.0045Y(0)Z(1)Y(2)Y(4)Z(5)Y(6)-0.0004Y(0)Z(1)Y(2)X(4)Z(5)X(6)-0.0004X(0)Z(1)X(2)Y(4)Z(5)Y(6)-0.0045X(0)Z(1)X(2)X(4)Z(5)X(6)-0.0041X(0)Z(1)Y(2)Y(4)Z(5)X(6)+0.0438Z(0)Z(3)+0.0005Y(0)Z(1)Y(2)Z(3)+0.0005X(0)Z(1)X(2)Z(3)-0.0068Y(0)Z(1)Z(2)Y(4)-0.0068X(0)Z(1)Z(2)X(4)-0.0149Y(0)Z(1)Z(2)Z(4)Z(5)Y(6)-0.0149X(0)Z(1)Z(2)Z(4)Z(5)X(6)+0.0149Z(0)Y(3)Z(4)Y(5)+0.0149Z(0)X(3)Z(4)X(5)+0.0025Y(0)Z(1)Y(2)Y(3)Z(4)Y(5)+0.0025Y(0)Z(1)Y(2)X(3)Z(4)X(5)+0.0025X(0)Z(1)X(2)Y(3)Z(4)Y(5)+0.0025X(0)Z(1)X(2)X(3)Z(4)X(5)+0.0090Y(0)Z(1)Z(2)X(3)X(4)Y(5)-0.0090Y(0)Z(1)Z(2)Y(3)X(4)X(5)-0.0090X(0)Z(1)Z(2)X(3)Y(4)Y(5)+0.0090X(0)Z(1)Z(2)Y(3)Y(4)X(5)-0.0081Y(0)Z(1)Z(2)X(3)X(5)Y(6)-0.0081Y(0)Z(1)Z(2)Y(3)Y(5)Y(6)-0.0081X(0)Z(1)Z(2)X(3)X(5)X(6)-0.0081X(0)Z(1)Z(2)Y(3)Y(5)X(6)+0.0068Z(0)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0068Z(0)X(3)Z(4)Z(5)Z(6)X(7)+0.0118Y(0)Z(1)Y(2)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0118Y(0)Z(1)Y(2)X(3)Z(4)Z(5)Z(6)X(7)+0.0118X(0)Z(1)X(2)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0118X(0)Z(1)X(2)X(3)Z(4)Z(5)Z(6)X(7)+0.0040Y(0)Z(1)Z(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0040Y(0)Z(1)Z(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0040X(0)Z(1)Z(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0040X(0)Z(1)Z(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0090Y(0)Z(1)Z(2)X(3)X(6)Y(7)-0.0090Y(0)Z(1)Z(2)Y(3)X(6)X(7)-0.0090X(0)Z(1)Z(2)X(3)Y(6)Y(7)+0.0090X(0)Z(1)Z(2)Y(3)Y(6)X(7)+0.0387Z(0)Z(4)+0.0069Z(0)Y(4)Z(5)Y(6)+0.0069Z(0)X(4)Z(5)X(6)+0.0033Y(0)Z(1)Y(2)Z(4)+0.0033X(0)Z(1)X(2)Z(4)+0.0032Y(0)Z(1)Z(2)Z(3)Z(5)Y(6)+0.0032X(0)Z(1)Z(2)Z(3)Z(5)X(6)+0.0661Z(0)Z(5)-0.0057Y(0)Z(1)Y(2)Z(5)-0.0057X(0)Z(1)X(2)Z(5)+0.0232Y(0)Z(1)Z(2)Z(3)Y(4)Z(5)+0.0232X(0)Z(1)Z(2)Z(3)X(4)Z(5)+0.0071Y(0)Z(1)Z(2)Z(3)Z(4)Y(6)+0.0071X(0)Z(1)Z(2)Z(3)Z(4)X(6)-0.0033Z(0)Y(5)Z(6)Y(7)-0.0033Z(0)X(5)Z(6)X(7)+0.0037Y(0)Z(1)Y(2)Y(5)Z(6)Y(7)+0.0037Y(0)Z(1)Y(2)X(5)Z(6)X(7)+0.0037X(0)Z(1)X(2)Y(5)Z(6)Y(7)+0.0037X(0)Z(1)X(2)X(5)Z(6)X(7)-0.0039Y(0)Z(1)Z(2)Z(3)Y(4)Y(5)Z(6)Y(7)-0.0039Y(0)Z(1)Z(2)Z(3)Y(4)X(5)Z(6)X(7)-0.0039X(0)Z(1)Z(2)Z(3)X(4)Y(5)Z(6)Y(7)-0.0039X(0)Z(1)Z(2)Z(3)X(4)X(5)Z(6)X(7)+0.0002Y(0)Z(1)Z(2)Z(3)Z(4)X(5)X(6)Y(7)-0.0002Y(0)Z(1)Z(2)Z(3)Z(4)Y(5)X(6)X(7)-0.0002X(0)Z(1)Z(2)Z(3)Z(4)X(5)Y(6)Y(7)+0.0002X(0)Z(1)Z(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.0306Z(0)Z(6)+0.0033Y(0)Z(1)Y(2)Z(6)+0.0033X(0)Z(1)X(2)Z(6)-0.0034Y(0)Z(1)Z(2)Z(3)Y(4)Z(6)-0.0034X(0)Z(1)Z(2)Z(3)X(4)Z(6)+0.0427Z(0)Z(7)-0.0057Y(0)Z(1)Y(2)Z(7)-0.0057X(0)Z(1)X(2)Z(7)-0.0035Y(0)Z(1)Z(2)Z(3)Y(4)Z(7)-0.0035X(0)Z(1)Z(2)Z(3)X(4)Z(7)-0.0144Y(0)Z(1)Z(2)Z(3)Z(4)Z(5)Y(6)Z(7)-0.0144X(0)Z(1)Z(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.0438Z(1)Z(2)+0.0005Y(1)Y(3)+0.0005X(1)X(3)-0.0068Y(1)Z(3)Z(4)Y(5)-0.0068X(1)Z(3)Z(4)X(5)-0.0149Y(1)Z(3)Z(4)Z(5)Z(6)Y(7)-0.0149X(1)Z(3)Z(4)Z(5)Z(6)X(7)+0.0149Z(1)Y(2)Z(3)Y(4)+0.0149Z(1)X(2)Z(3)X(4)+0.0025Y(1)X(2)X(3)Y(4)-0.0025Y(1)Y(2)X(3)X(4)-0.0025X(1)X(2)Y(3)Y(4)+0.0025X(1)Y(2)Y(3)X(4)-0.0090Y(1)X(2)X(4)Y(5)-0.0090Y(1)Y(2)Y(4)Y(5)-0.0090X(1)X(2)X(4)X(5)-0.0090X(1)Y(2)Y(4)X(5)-0.0081Y(1)X(2)X(4)Z(5)Z(6)Y(7)-0.0081Y(1)Y(2)Y(4)Z(5)Z(6)Y(7)-0.0081X(1)X(2)X(4)Z(5)Z(6)X(7)-0.0081X(1)Y(2)Y(4)Z(5)Z(6)X(7)+0.0068Z(1)Y(2)Z(3)Z(4)Z(5)Y(6)+0.0068Z(1)X(2)Z(3)Z(4)Z(5)X(6)+0.0118Y(1)X(2)X(3)Z(4)Z(5)Y(6)-0.0118Y(1)Y(2)X(3)Z(4)Z(5)X(6)-0.0118X(1)X(2)Y(3)Z(4)Z(5)Y(6)+0.0118X(1)Y(2)Y(3)Z(4)Z(5)X(6)+0.0040Y(1)X(2)X(5)Y(6)-0.0040Y(1)Y(2)X(5)X(6)-0.0040X(1)X(2)Y(5)Y(6)+0.0040X(1)Y(2)Y(5)X(6)-0.0090Y(1)X(2)X(6)Y(7)-0.0090Y(1)Y(2)Y(6)Y(7)-0.0090X(1)X(2)X(6)X(7)-0.0090X(1)Y(2)Y(6)X(7)+0.0301Z(1)Z(3)+0.0031Z(1)Y(3)Z(4)Y(5)+0.0031Z(1)X(3)Z(4)X(5)+0.0043Z(1)Y(3)Z(4)Z(5)Z(6)Y(7)+0.0043Z(1)X(3)Z(4)Z(5)Z(6)X(7)-0.0043Y(1)Z(2)Z(4)Y(5)-0.0043X(1)Z(2)Z(4)X(5)-0.0031Y(1)Z(2)Z(4)Z(5)Z(6)Y(7)-0.0031X(1)Z(2)Z(4)Z(5)Z(6)X(7)-0.0041Y(1)Z(2)X(3)X(5)Z(6)Y(7)-0.0045Y(1)Z(2)Y(3)Y(5)Z(6)Y(7)-0.0004Y(1)Z(2)Y(3)X(5)Z(6)X(7)-0.0004X(1)Z(2)X(3)Y(5)Z(6)Y(7)-0.0045X(1)Z(2)X(3)X(5)Z(6)X(7)-0.0041X(1)Z(2)Y(3)Y(5)Z(6)X(7)+0.0661Z(1)Z(4)-0.0057Y(1)Z(2)Y(3)Z(4)-0.0057X(1)Z(2)X(3)Z(4)+0.0232Y(1)Z(2)Z(3)Y(5)+0.0232X(1)Z(2)Z(3)X(5)+0.0071Y(1)Z(2)Z(3)Z(5)Z(6)Y(7)+0.0071X(1)Z(2)Z(3)Z(5)Z(6)X(7)-0.0033Z(1)Y(4)Z(5)Y(6)-0.0033Z(1)X(4)Z(5)X(6)+0.0037Y(1)Z(2)Y(3)Y(4)Z(5)Y(6)+0.0037Y(1)Z(2)Y(3)X(4)Z(5)X(6)+0.0037X(1)Z(2)X(3)Y(4)Z(5)Y(6)+0.0037X(1)Z(2)X(3)X(4)Z(5)X(6)-0.0039Y(1)Z(2)Z(3)X(4)X(5)Y(6)+0.0039Y(1)Z(2)Z(3)Y(4)X(5)X(6)+0.0039X(1)Z(2)Z(3)X(4)Y(5)Y(6)-0.0039X(1)Z(2)Z(3)Y(4)Y(5)X(6)-0.0002Y(1)Z(2)Z(3)X(4)X(6)Y(7)-0.0002Y(1)Z(2)Z(3)Y(4)Y(6)Y(7)-0.0002X(1)Z(2)Z(3)X(4)X(6)X(7)-0.0002X(1)Z(2)Z(3)Y(4)Y(6)X(7)+0.0387Z(1)Z(5)+0.0069Z(1)Y(5)Z(6)Y(7)+0.0069Z(1)X(5)Z(6)X(7)+0.0033Y(1)Z(2)Y(3)Z(5)+0.0033X(1)Z(2)X(3)Z(5)+0.0032Y(1)Z(2)Z(3)Z(4)Z(6)Y(7)+0.0032X(1)Z(2)Z(3)Z(4)Z(6)X(7)+0.0427Z(1)Z(6)-0.0057Y(1)Z(2)Y(3)Z(6)-0.0057X(1)Z(2)X(3)Z(6)-0.0035Y(1)Z(2)Z(3)Z(4)Y(5)Z(6)-0.0035X(1)Z(2)Z(3)Z(4)X(5)Z(6)-0.0144Y(1)Z(2)Z(3)Z(4)Z(5)Y(7)-0.0144X(1)Z(2)Z(3)Z(4)Z(5)X(7)+0.0306Z(1)Z(7)+0.0033Y(1)Z(2)Y(3)Z(7)+0.0033X(1)Z(2)X(3)Z(7)-0.0034Y(1)Z(2)Z(3)Z(4)Y(5)Z(7)-0.0034X(1)Z(2)Z(3)Z(4)X(5)Z(7)+0.0660Z(2)Z(3)-0.0061Y(2)Y(4)-0.0061X(2)X(4)-0.0149Y(2)Z(4)Z(5)Y(6)-0.0149X(2)Z(4)Z(5)X(6)-0.0061Z(2)Y(3)Z(4)Y(5)-0.0061Z(2)X(3)Z(4)X(5)+0.0121Y(2)X(3)X(4)Y(5)-0.0121Y(2)Y(3)X(4)X(5)-0.0121X(2)X(3)Y(4)Y(5)+0.0121X(2)Y(3)Y(4)X(5)-0.0102Y(2)X(3)X(5)Y(6)-0.0102Y(2)Y(3)Y(5)Y(6)-0.0102X(2)X(3)X(5)X(6)-0.0102X(2)Y(3)Y(5)X(6)-0.0149Z(2)Y(3)Z(4)Z(5)Z(6)Y(7)-0.0149Z(2)X(3)Z(4)Z(5)Z(6)X(7)+0.0102Y(2)X(3)X(4)Z(5)Z(6)Y(7)-0.0102Y(2)Y(3)X(4)Z(5)Z(6)X(7)-0.0102X(2)X(3)Y(4)Z(5)Z(6)Y(7)+0.0102X(2)Y(3)Y(4)Z(5)Z(6)X(7)+0.0274Y(2)X(3)X(6)Y(7)-0.0274Y(2)Y(3)X(6)X(7)-0.0274X(2)X(3)Y(6)Y(7)+0.0274X(2)Y(3)Y(6)X(7)+0.0306Z(2)Z(4)+0.0069Z(2)Y(4)Z(5)Y(6)+0.0069Z(2)X(4)Z(5)X(6)+0.0034Y(2)Z(3)Z(5)Y(6)+0.0034X(2)Z(3)Z(5)X(6)+0.0427Z(2)Z(5)+0.0144Y(2)Z(3)Y(4)Z(5)+0.0144X(2)Z(3)X(4)Z(5)+0.0035Y(2)Z(3)Z(4)Y(6)+0.0035X(2)Z(3)Z(4)X(6)-0.0033Z(2)Y(5)Z(6)Y(7)-0.0033Z(2)X(5)Z(6)X(7)-0.0002Y(2)Z(3)Y(4)Y(5)Z(6)Y(7)-0.0002Y(2)Z(3)Y(4)X(5)Z(6)X(7)-0.0002X(2)Z(3)X(4)Y(5)Z(6)Y(7)-0.0002X(2)Z(3)X(4)X(5)Z(6)X(7)+0.0039Y(2)Z(3)Z(4)X(5)X(6)Y(7)-0.0039Y(2)Z(3)Z(4)Y(5)X(6)X(7)-0.0039X(2)Z(3)Z(4)X(5)Y(6)Y(7)+0.0039X(2)Z(3)Z(4)Y(5)Y(6)X(7)+0.0387Z(2)Z(6)-0.0032Y(2)Z(3)Y(4)Z(6)-0.0032X(2)Z(3)X(4)Z(6)+0.0661Z(2)Z(7)-0.0071Y(2)Z(3)Y(4)Z(7)-0.0071X(2)Z(3)X(4)Z(7)-0.0232Y(2)Z(3)Z(4)Z(5)Y(6)Z(7)-0.0232X(2)Z(3)Z(4)Z(5)X(6)Z(7)+0.0427Z(3)Z(4)+0.0144Y(3)Y(5)+0.0144X(3)X(5)+0.0035Y(3)Z(5)Z(6)Y(7)+0.0035X(3)Z(5)Z(6)X(7)-0.0033Z(3)Y(4)Z(5)Y(6)-0.0033Z(3)X(4)Z(5)X(6)-0.0002Y(3)X(4)X(5)Y(6)+0.0002Y(3)Y(4)X(5)X(6)+0.0002X(3)X(4)Y(5)Y(6)-0.0002X(3)Y(4)Y(5)X(6)-0.0039Y(3)X(4)X(6)Y(7)-0.0039Y(3)Y(4)Y(6)Y(7)-0.0039X(3)X(4)X(6)X(7)-0.0039X(3)Y(4)Y(6)X(7)+0.0306Z(3)Z(5)+0.0069Z(3)Y(5)Z(6)Y(7)+0.0069Z(3)X(5)Z(6)X(7)+0.0034Y(3)Z(4)Z(6)Y(7)+0.0034X(3)Z(4)Z(6)X(7)+0.0661Z(3)Z(6)-0.0071Y(3)Z(4)Y(5)Z(6)-0.0071X(3)Z(4)X(5)Z(6)-0.0232Y(3)Z(4)Z(5)Y(7)-0.0232X(3)Z(4)Z(5)X(7)+0.0387Z(3)Z(7)-0.0032Y(3)Z(4)Y(5)Z(7)-0.0032X(3)Z(4)X(5)Z(7)+0.0733Z(4)Z(5)-0.0047Y(4)Y(6)-0.0047X(4)X(6)-0.0047Z(4)Y(5)Z(6)Y(7)-0.0047Z(4)X(5)Z(6)X(7)+0.0019Y(4)X(5)X(6)Y(7)-0.0019Y(4)Y(5)X(6)X(7)-0.0019X(4)X(5)Y(6)Y(7)+0.0019X(4)Y(5)Y(6)X(7)+0.0393Z(4)Z(6)+0.0413Z(4)Z(7)-0.0047Y(4)Z(5)Y(6)Z(7)-0.0047X(4)Z(5)X(6)Z(7)+0.0413Z(5)Z(6)-0.0047Y(5)Y(7)-0.0047X(5)X(7)+0.0393Z(5)Z(7)+0.0733Z(6)Z(7)
"""

pattern = r"finished after ([\d\.]+)s\n\nmolecule: (\w+), distance: ([\d\.]+)\n(.+?)(?=\nfinished after|\Z)"
matches = re.findall(pattern, data, re.DOTALL)

result = [(float(t), mol, float(d), parse_hamiltonian(hamiltonian.strip())) for t, mol, d, hamiltonian in matches]

print(result)

[(21.832292079925537, 'H2', 0.1, SparsePauliOp(['IIII', 'IIIZ', 'IYZY', 'IXZX', 'IIZI', 'YZYI', 'XZXI', 'IZII', 'ZIII', 'IIZZ', 'IYIY', 'IXIX', 'YZYZ', 'XZXZ', 'YXXY', 'XXYY', 'YYXX', 'XYYX', 'IZIZ', 'ZIIZ', 'ZYZY', 'ZXZX', 'IZZI', 'YIYI', 'XIXI', 'ZIZI', 'ZZII'],
              coeffs=[ 4.9081+0.j,  0.2909+0.j, -0.0056+0.j, -0.0056+0.j,  0.2909+0.j,
 -0.0056+0.j, -0.0056+0.j, -0.7411+0.j, -0.7411+0.j,  0.2481+0.j,
  0.0781+0.j,  0.0781+0.j,  0.0781+0.j,  0.0781+0.j,  0.0629+0.j,
 -0.0629+0.j, -0.0629+0.j,  0.0629+0.j,  0.1693+0.j,  0.2322+0.j,
  0.0836+0.j,  0.0836+0.j,  0.2322+0.j,  0.0836+0.j,  0.0836+0.j,
  0.1693+0.j,  0.2329+0.j])), (35.75117349624634, 'LiH', 0.1, SparsePauliOp(['IIII', 'IIIZ', 'IYZY', 'IXZX', 'IIZI', 'YZYI', 'XZXI', 'IZII', 'ZIII', 'IIZZ', 'IYIY', 'IXIX', 'YZYZ', 'XZXZ', 'YXXY', 'XXYY', 'YYXX', 'XYYX', 'IZIZ', 'ZIIZ', 'ZYZY', 'ZXZX', 'IZZI', 'YIYI', 'XIXI', 'ZIZI', 'ZZII'],
              coeffs=[ 2.5983e+00+0.j,  9.8600e-02+0.j, -0.0000e+00+0.j, -0.0000e+00+0.j,


# Now ansatze example and example hamiltonian

In [10]:
# example
ansatz_example_text="""
finished after 21.994049310684204s

molecule: H2, distance: 0.1, k: 1
circuit: 
X(target=(0,))
Ry(target=(2,), parameter=(((0, 1),), 'D', (None, 0)))
X(target=(0,), control=(2,))
X(target=(1,), control=(0,))
X(target=(3,), control=(2,))
FermionicExcitation(target=(0, 1, 2), control=(), parameter=((0, 1), 'S', (None, 0)))
FermionicExcitation(target=(1, 2, 3), control=(), parameter=((0, 1), 'S', (None, 0)))


"""

In [11]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RYGate, MCXGate
from qiskit_nature.second_q.circuit.library import FermionicExcitation
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms.optimizers import L_BFGS_B
import numpy as np
import time

theta1 = Parameter("θ1")
theta2 = Parameter("θ2")


qc = QuantumCircuit(4)

qc.x(0)
qc.ry(theta1, 2)
qc.cx(2, 0)
qc.cx(0, 1)
qc.cx(2, 3)

qc.append(FermionicExcitation(3, theta1), [0, 1, 2])
qc.append(FermionicExcitation(3, theta2), [1, 2, 3])


ImportError: cannot import name 'FermionicExcitation' from 'qiskit_nature.second_q.circuit.library' (/home/home/Desktop/PLAYGROUND/vqe-analysis-paper/qiskit-env/lib/python3.13/site-packages/qiskit_nature/second_q/circuit/library/__init__.py)

In [ ]:
hamiltonian_tuple = (21.832292079925537, 'H2', 0.1, SparsePauliOp(['IIII', 'IIIZ', 'IYZY', 'IXZX', 'IIZI', 'YZYI', 'XZXI', 'IZII', 'ZIII', 'IIZZ', 'IYIY', 'IXIX', 'YZYZ', 'XZXZ', 'YXXY', 'XXYY', 'YYXX', 'XYYX', 'IZIZ', 'ZIIZ', 'ZYZY', 'ZXZX', 'IZZI', 'YIYI', 'XIXI', 'ZIZI', 'ZZII'],
              coeffs=[ 4.9081+0.j,  0.2909+0.j, -0.0056+0.j, -0.0056+0.j,  0.2909+0.j,
 -0.0056+0.j, -0.0056+0.j, -0.7411+0.j, -0.7411+0.j,  0.2481+0.j,
  0.0781+0.j,  0.0781+0.j,  0.0781+0.j,  0.0781+0.j,  0.0629+0.j,
 -0.0629+0.j, -0.0629+0.j,  0.0629+0.j,  0.1693+0.j,  0.2322+0.j,
  0.0836+0.j,  0.0836+0.j,  0.2322+0.j,  0.0836+0.j,  0.0836+0.j,
  0.1693+0.j,  0.2329+0.j]))

In [ ]:
eiterations = []
eruntimes = []
eenergies = []
estimator = StatevectorEstimator()
optimizer = L_BFGS_B(maxiter=1000)
def estimate_energy(parameters):
    job = estimator.run(
        [(qc, hamiltonian_tuple[3], parameters)]
    )
    result = job.result()[0].data.evs
    eiterations.append(result)
    return result

initial_point = np.zeros(qc.num_parameters)
start = time.perf_counter()
result = optimizer.minimize(fun=estimate_energy, x0=initial_point)
end = time.perf_counter()
eruntimes.append(end - start)
eenergies.append(result.fun)